# Load and Preprocess

In [45]:
import pandas as pd
# Remove the limit on the number of columns displayed
pd.set_option('display.max_columns', None)

In [46]:
table_df = pd.read_csv("finaltable1.csv")

In [47]:
table_df.tail()

,Unnamed: 0,Match ID,Home Team,Away Team,Home Score,Away Score,Winner,Prev Home Played,Prev Home Won,Prev Home Drawn,Prev Home Lost,Prev Home Goals Scored,Prev Home Goals Conceded,Prev Home Goal Difference,Prev Home Points,Prev Home Form,Prev Away Played,Prev Away Won,Prev Away Drawn,Prev Away Lost,Prev Away Goals Scored,Prev Away Goals Conceded,Prev Away Goal Difference,Prev Away Points,Prev Away Form,Date,Time,Wk,Day,xG,Score,xG.1,Attendance,Venue,Referee,Away Win Odds,Home Win Odds,Draw Odds,StadiumID,Latitude,Longitude,Datetime,Temperature,Precipitation,H2H_Home_Total_Wins_Last_4,H2H_Away_Total_Wins_Last_4,H2H_Draws_Last_4,Composite Key,away_team_code,home_team_code,home_xg,away_xg,is_result,has_data,url,away_points,away_expected_points,away_np_xg,away_np_xg_difference,away_ppda,away_deep_completions,home_points,home_expected_points,home_np_xg,home_np_xg_difference,home_ppda,home_deep_completions,Unnamed: 0_x,gameweek,home_team_fdr,away_team_fdr,home_team,away_team,composite_key,HomeTeam,AwayTeam,B365H,B365D,B365A,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,Home_player_average,Away_player_average
3,3,497447,Southampton,Manchester Utd,0.0,3.0,AWAY_TEAM,3,0,0,3,1,5,-4,0,0.0,3,1,0,2,2,5,-3,3,1.0,2024-09-14,11:30,4,Sat,1.1,0–3,2.6,31144.0,St Mary's Stadium,Stuart Attwell,NaN,NaN,NaN,15.0,50.9058,-1.3913,2024-09-14T11:30:00,16.136499,0.0,0,1,0,2024-09-14_Southampton_Manchester Utd,MUN,SOU,1.355930,3.026930,True,True,https://understat.com/match/26632,3.0,2.5032,3.026930,2.481613,10.344828,17.0,0.0,0.3435,0.545317,-2.481613,10.857143,3.0,31,4.0,3,2,Southampton,Manchester Utd,2024-09-14_Southampton_Manchester Utd,Southampton,Manchester Utd,4.50,4.00,1.70,4.0,10.0,11.0,14.0,0.0,7.0,1.0,4.0,1.0,0.0,8.010871,11.243967
4,4,497469,Wolves,Liverpool,1.0,2.0,AWAY_TEAM,5,0,1,4,5,14,-9,1,0.2,5,4,0,1,10,1,9,12,2.4,2024-09-28,16:30,6,Sat,0.6,1–2,2.5,31413.0,Molineux Stadium,Anthony Taylor,NaN,NaN,NaN,19.0,52.5903,-2.1303,2024-09-28T16:30:00,11.878500,0.0,0,3,0,2024-09-28_Wolves_Liverpool,LIV,WOL,0.748925,2.117240,True,True,https://understat.com/match/26658,3.0,2.4807,1.356070,0.607145,14.777778,11.0,0.0,0.3471,0.748925,-0.607145,14.000000,2.0,57,6.0,5,2,Wolves,Liverpool,2024-09-28_Wolves_Liverpool,Wolves,Liverpool,8.00,5.75,1.33,3.0,6.0,16.0,8.0,2.0,10.0,2.0,3.0,0.0,0.0,5.327128,11.979388
5,5,497519,Wolves,Southampton,2.0,0.0,HOME_TEAM,10,0,3,7,14,27,-13,3,0.4,10,1,1,8,7,19,-12,4,0.6,2024-11-09,15:00,11,Sat,1.3,2–0,0.6,31403.0,Molineux Stadium,Thomas Bramall,NaN,NaN,NaN,19.0,52.5903,-2.1303,2024-11-09T15:00:00,7.128500,0.0,0,0,0,2024-11-09_Wolves_Southampton,SOU,WOL,0.912781,0.792655,True,True,https://understat.com/match/26711,0.0,1.1777,0.792655,-0.120126,5.814815,6.0,3.0,1.4630,0.912781,0.120126,25.437500,7.0,104,11.0,2,2,Wolves,Southampton,2024-11-09_Wolves_Southampton,Wolves,Southampton,1.91,3.75,3.75,4.0,0.0,14.0,12.0,1.0,9.0,1.0,4.0,0.0,0.0,12.271554,9.996454
6,6,497541,Arsenal,Manchester Utd,2.0,0.0,HOME_TEAM,13,7,4,2,26,14,12,25,1.6,13,5,4,4,17,13,4,19,1.6,2024-12-04,20:15,14,Wed,2.1,2–0,0.2,60256.0,Emirates Stadium,Samuel Barrott,NaN,NaN,NaN,3.0,51.5550,-0.1086,2024-12-04T20:15:00,NaN,NaN,1,2,0,2024-12-04_Arsenal_Manchester Utd,MUN,ARS,2.966840,0.256083,True,True,https://understat.com/match/26733,0.0,0.0343,0.256083,-2.710757,19.400000,1.0,3.0,2.9395,2.966840,2.710757,11.800000,8.0,137,14.0,3,5,Arsenal,Manchester Utd,2024-12-04_Arsenal_Manchester Utd,Arsenal,Manchester Utd,1.44,4.50,7.00,6.0,2.0,12.0,8.0,13.0,0.0,1.0,3.0,0.0,0.0,8.972388,5.079682
7,7,497577,Manchester Utd,Bournemouth,NaN,NaN,NaN,15,5,4,6,19,18,1,19,1.4,15,7,3,5,23,20,3,24,1.8,2024-12-22,14:00,17,Sun,NaN,NaN,NaN,NaN,Old Trafford,NaN,NaN,NaN,NaN,1.0,53.4631,-2.2914,2024-12-22T14:00:00,NaN,NaN,2,0,0,2024-12-22_Manchester Utd_Bournemouth,BOU,MUN,NaN,NaN,False,False,https://understat.com/match/26769,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,168,17.0,2,3,Manchester Utd,Bournemouth,2024-12-22_Manchester Utd_Bournemouth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9

In [48]:
len(table_df)

8

# Data Preprocessing

## Handling Missing Values

In [51]:
def check_missing_values():
    # Check for missing values in the dataset
    missing_values = table_df.isnull().sum()
    
    # Filter for columns with missing values
    missing_values = missing_values[missing_values > 0]
    
    # Display missing values
    print("Columns with Missing Values:")
    print(missing_values)


In [52]:
check_missing_values()

Columns with Missing Values:
Home Score               1
Away Score               1
Winner                   1
xG                       1
Score                    1
xG.1                     1
Attendance               1
Referee                  1
Away Win Odds            8
Home Win Odds            8
Draw Odds                8
Temperature              2
Precipitation            2
home_xg                  1
away_xg                  1
away_points              1
away_expected_points     1
away_np_xg               1
away_np_xg_difference    1
away_ppda                1
away_deep_completions    1
home_points              1
home_expected_points     1
home_np_xg               1
home_np_xg_difference    1
home_ppda                1
home_deep_completions    1
HomeTeam                 1
AwayTeam                 1
B365H                    1
B365D                    1
B365A                    1
HST                      1
AST                      1
HF                       1
AF                       1

Store null fixtures for using to prediction later

In [54]:
# Create a new DataFrame with rows where 'Home Score' or 'Away Score' is null
null_scores_df = table_df[table_df['Home Score'].isnull() | table_df['Away Score'].isnull()]


In [55]:
null_scores_df.head()

,Unnamed: 0,Match ID,Home Team,Away Team,Home Score,Away Score,Winner,Prev Home Played,Prev Home Won,Prev Home Drawn,Prev Home Lost,Prev Home Goals Scored,Prev Home Goals Conceded,Prev Home Goal Difference,Prev Home Points,Prev Home Form,Prev Away Played,Prev Away Won,Prev Away Drawn,Prev Away Lost,Prev Away Goals Scored,Prev Away Goals Conceded,Prev Away Goal Difference,Prev Away Points,Prev Away Form,Date,Time,Wk,Day,xG,Score,xG.1,Attendance,Venue,Referee,Away Win Odds,Home Win Odds,Draw Odds,StadiumID,Latitude,Longitude,Datetime,Temperature,Precipitation,H2H_Home_Total_Wins_Last_4,H2H_Away_Total_Wins_Last_4,H2H_Draws_Last_4,Composite Key,away_team_code,home_team_code,home_xg,away_xg,is_result,has_data,url,away_points,away_expected_points,away_np_xg,away_np_xg_difference,away_ppda,away_deep_completions,home_points,home_expected_points,home_np_xg,home_np_xg_difference,home_ppda,home_deep_completions,Unnamed: 0_x,gameweek,home_team_fdr,away_team_fdr,home_team,away_team,composite_key,HomeTeam,AwayTeam,B365H,B365D,B365A,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,Home_player_average,Away_player_average
7,7,497577,Manchester Utd,Bournemouth,NaN,NaN,NaN,15,5,4,6,19,18,1,19,1.4,15,7,3,5,23,20,3,24,1.8,2024-12-22,14:00,17,Sun,NaN,NaN,NaN,NaN,Old Trafford,NaN,NaN,NaN,NaN,1.0,53.4631,-2.2914,2024-12-22T14:00:00,NaN,NaN,2,0,0,2024-12-22_Manchester Utd_Bournemouth,BOU,MUN,NaN,NaN,False,False,https://understat.com/match/26769,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,168,17.0,2,3,Manchester Utd,Bournemouth,2024-12-22_Manchester Utd_Bournemouth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.035769,9.326714


Remove matches that have not been played..

In [57]:
table_df = table_df.dropna(subset=['Home Score', 'Away Score'])
# Reset the index after filtering
table_df.reset_index(drop=True, inplace=True)

In [58]:
check_missing_values()

Columns with Missing Values:
Away Win Odds    7
Home Win Odds    7
Draw Odds        7
Temperature      1
Precipitation    1
dtype: int64


In [59]:
# Fill missing 'xG' values with corresponding 'home_xg' values for the same row
table_df['xG'] = table_df.apply(
    lambda row: row['home_xg'] if pd.isnull(row['xG']) else row['xG'],
    axis=1
)
table_df['xG.1'] = table_df.apply(
    lambda row: row['away_xg'] if pd.isnull(row['xG.1']) else row['xG.1'],
    axis=1
)

# Verify that there are no more missing values in 'xG'
missing_xg_after = table_df['xG'].isnull().sum()
print(f"Remaining missing values in 'xG': {missing_xg_after}")
missing_xg_after = table_df['xG.1'].isnull().sum()
print(f"Remaining missing values in 'xG.1': {missing_xg_after}")


Remaining missing values in 'xG': 0
Remaining missing values in 'xG.1': 0


In [60]:
check_missing_values()

Columns with Missing Values:
Away Win Odds    7
Home Win Odds    7
Draw Odds        7
Temperature      1
Precipitation    1
dtype: int64


Drop the score column because we already have home scores and away scores

In [62]:
# Drop the 'Score' column from the dataset
table_df.drop(columns=['Score','Precipitation'], inplace=True)

In [63]:
check_missing_values()

Columns with Missing Values:
Away Win Odds    7
Home Win Odds    7
Draw Odds        7
Temperature      1
dtype: int64


In [64]:
# Fill missing 'Attendance' values with the median attendance
median_attendance = table_df['Attendance'].median()
table_df['Attendance'].fillna(median_attendance, inplace=True)

# Verify there are no missing values in 'Attendance'
print(f"Remaining missing values in 'Attendance': {table_df['Attendance'].isnull().sum()}")


Remaining missing values in 'Attendance': 0


In [65]:
# Filter rows where 'Referee' is null
rows_with_missing_referee = table_df[table_df['Referee'].isnull()]
rows_with_missing_referee.head(7)

,Unnamed: 0,Match ID,Home Team,Away Team,Home Score,Away Score,Winner,Prev Home Played,Prev Home Won,Prev Home Drawn,Prev Home Lost,Prev Home Goals Scored,Prev Home Goals Conceded,Prev Home Goal Difference,Prev Home Points,Prev Home Form,Prev Away Played,Prev Away Won,Prev Away Drawn,Prev Away Lost,Prev Away Goals Scored,Prev Away Goals Conceded,Prev Away Goal Difference,Prev Away Points,Prev Away Form,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,Away Win Odds,Home Win Odds,Draw Odds,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Home_Total_Wins_Last_4,H2H_Away_Total_Wins_Last_4,H2H_Draws_Last_4,Composite Key,away_team_code,home_team_code,home_xg,away_xg,is_result,has_data,url,away_points,away_expected_points,away_np_xg,away_np_xg_difference,away_ppda,away_deep_completions,home_points,home_expected_points,home_np_xg,home_np_xg_difference,home_ppda,home_deep_completions,Unnamed: 0_x,gameweek,home_team_fdr,away_team_fdr,home_team,away_team,composite_key,HomeTeam,AwayTeam,B365H,B365D,B365A,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,Home_player_average,Away_player_average


In [66]:
# Fill missing 'Referee' values with the mode (most common referee)
mode_referee = table_df['Referee'].mode()[0]
table_df['Referee'].fillna(mode_referee, inplace=True)

# Verify there are no missing values in 'Referee'
print(f"Remaining missing values in 'Referee': {table_df['Referee'].isnull().sum()}")


Remaining missing values in 'Referee': 0


In [67]:
check_missing_values()

Columns with Missing Values:
Away Win Odds    7
Home Win Odds    7
Draw Odds        7
Temperature      1
dtype: int64


In [68]:
from sklearn.impute import KNNImputer
import pandas as pd

# Load the dataset (assuming it's in table_df)
# Replace 'Temperature' and any other relevant columns for KNN imputation
columns_to_impute = ['Temperature']  # Include columns correlated with Temperature

# Select only the relevant columns for imputation
impute_df = table_df[columns_to_impute]

# Initialize the KNN imputer
knn_imputer = KNNImputer(n_neighbors=5)  # Use 5 neighbors by default

# Perform KNN imputation
imputed_data = knn_imputer.fit_transform(impute_df)

# Replace the original 'Temperature' column with the imputed values
table_df['Temperature'] = imputed_data[:, 0]

# Verify the imputation
print("Remaining missing values in 'Temperature':", table_df['Temperature'].isnull().sum())


Remaining missing values in 'Temperature': 0


In [69]:
check_missing_values()

Columns with Missing Values:
Away Win Odds    7
Home Win Odds    7
Draw Odds        7
dtype: int64


In [70]:
columns_to_remove = ['Away Win Odds', 'Home Win Odds', 'Draw Odds', 'Unnamed: 0']
table_df.drop(columns=columns_to_remove, inplace=True)


In [71]:
check_missing_values()

Columns with Missing Values:
Series([], dtype: int64)


## Check for outliers

In [73]:
# Function to identify outliers based on IQR
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)  # First quartile
    Q3 = df[column].quantile(0.75)  # Third quartile
    IQR = Q3 - Q1  # Interquartile range
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

# Check for outliers in numerical columns
numerical_columns = table_df.select_dtypes(include=['float64', 'int64']).columns
outliers = {col: detect_outliers_iqr(table_df, col) for col in numerical_columns}

# Display the columns with outliers and their counts
outlier_counts = {col: len(outliers[col]) for col in outliers if len(outliers[col]) > 0}
print("Columns with outliers and their counts:")
print(outlier_counts)


Columns with outliers and their counts:
{'Prev Home Won': 1, 'Prev Home Goals Scored': 1, 'Prev Home Goal Difference': 1, 'Prev Home Points': 1, 'Prev Away Drawn': 1, 'Prev Away Lost': 1, 'Prev Away Goal Difference': 1, 'H2H_Home_Total_Wins_Last_4': 1, 'home_xg': 1, 'away_ppda': 2, 'home_ppda': 1, 'home_deep_completions': 1, 'B365H': 1, 'B365D': 1, 'B365A': 1, 'AST': 1, 'HY': 1, 'HR': 1}


Append the first 10 upcoming fixtures

In [75]:
# Append the first 10 rows of null_scores_df to table_df using concat
table_df = pd.concat([table_df, null_scores_df.head(10)], ignore_index=True)

In [76]:
table_df.tail(12)

,Match ID,Home Team,Away Team,Home Score,Away Score,Winner,Prev Home Played,Prev Home Won,Prev Home Drawn,Prev Home Lost,Prev Home Goals Scored,Prev Home Goals Conceded,Prev Home Goal Difference,Prev Home Points,Prev Home Form,Prev Away Played,Prev Away Won,Prev Away Drawn,Prev Away Lost,Prev Away Goals Scored,Prev Away Goals Conceded,Prev Away Goal Difference,Prev Away Points,Prev Away Form,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Home_Total_Wins_Last_4,H2H_Away_Total_Wins_Last_4,H2H_Draws_Last_4,Composite Key,away_team_code,home_team_code,home_xg,away_xg,is_result,has_data,url,away_points,away_expected_points,away_np_xg,away_np_xg_difference,away_ppda,away_deep_completions,home_points,home_expected_points,home_np_xg,home_np_xg_difference,home_ppda,home_deep_completions,Unnamed: 0_x,gameweek,home_team_fdr,away_team_fdr,home_team,away_team,composite_key,HomeTeam,AwayTeam,B365H,B365D,B365A,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,Home_player_average,Away_player_average,Unnamed: 0,Score,Away Win Odds,Home Win Odds,Draw Odds,Precipitation
0,497410,Manchester Utd,Fulham,1.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-16,19:00,1,Fri,2.4,0.4,73297.0,Old Trafford,Robert Jones,1.0,53.4631,-2.2914,2024-08-16T19:00:00,14.313000,1,0,0,2024-08-16_Manchester Utd_Fulham,FLH,MUN,2.042680,0.418711,True,True,https://understat.com/match/26602,0.0,0.2815,0.418711,-1.623969,10.833333,3.0,3.0,2.5696,2.042680,1.623969,7.379310,7.0,1,1.0,2,3,Manchester Utd,Fulham,2024-08-16_Manchester Utd_Fulham,Manchester Utd,Fulham,1.60,4.20,5.25,5.0,2.0,12.0,10.0,7.0,8.0,2.0,3.0,0.0,0.0,9.143500,8.974790,NaN,NaN,NaN,NaN,NaN,NaN
1,497412,Arsenal,Wolves,2.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00,1,Sat,1.2,0.5,60261.0,Emirates Stadium,Jarred Gillett,3.0,51.5550,-0.1086,2024-08-17T14:00:00,21.734999,3,0,0,2024-08-17_Arsenal_Wolves,WOL,ARS,1.628300,0.575835,True,True,https://understat.com/match/26604,0.0,0.5796,0.575835,-1.052465,10.818182,2.0,3.0,2.1894,1.628300,1.052465,7.769231,14.0,3,1.0,2,5,Arsenal,Wolves,2024-08-17_Arsenal_Wolves,Arsenal,Wolves,1.18,7.50,13.00,6.0,3.0,17.0,14.0,8.0,2.0,2.0,2.0,0.0,0.0,10.830699,6.027272,NaN,NaN,NaN,NaN,NaN,NaN
2,497436,Manchester Utd,Liverpool,0.0,3.0,AWAY_TEAM,2,1,0,1,2,2,0,3,1.5,2,2,0,0,4,0,4,6,3.0,2024-09-01,15:00,3,Sun,1.4,1.8,73738.0,Old Trafford,Anthony Taylor,1.0,53.4631,-2.2914,2024-09-01T15:00:00,19.213001,1,2,0,2024-09-01_Manchester Utd_Liverpool,LIV,MUN,1.500870,2.103970,True,True,https://understat.com/match/26631,3.0,1.7849,2.103970,0.603100,11.692308,9.0,0.0,0.9689,1.500870,-0.603100,12.500000,6.0,30,3.0,5,3,Manchester Utd,Liverpool,2024-09-01_Manchester Utd_Liverpool,Manchester Utd,Liverpool,3.70,4.20,1.85,3.0,3.0,7.0,7.0,5.0,2.0,4.0,1.0,0.0,0.0,4.950962,9.905478,NaN,NaN,NaN,NaN,NaN,NaN
3,497447,Southampton,Manchester Utd,0.0,3.0,AWAY_TEAM,3,0,0,3,1,5,-4,0,0.0,3,1,0,2,2,5,-3,3,1.0,2024-09-14,11:30,4,Sat,1.1,2.6,31144.0,St Mary's Stadium,Stuart Attwell,15.0,50.9058,-1.3913,2024-09-14T11:30:00,16.136499,0,1,0,2024-09-14_Southampton_Manchester Utd,MUN,SOU,1.355930,3.026930,True,True,https://understat.com/match/26632,3.0,2.5032,3.026930,2.481613,10.344828,17.0,0.0,0.3435,0.545317,-2.481613,10.857143,3.0,31,4.0,3,2,Southampton,Manchester Utd,2024-09-14_Southampton_Manchester Utd,Southampton,Manchester Utd,4.50,4.00,1.70,4.0,10.0,11.0,14.0,0.0,7.0,1.0,4.0,1.0,0.0,8.010871,11.243967,NaN,NaN,NaN,NaN,NaN,NaN
4,497469,Wolves,Liverpool,1.0,2.0,AWAY_TEAM,5,0,1,4,5,14,-9,1,0.2,5,4,0,1,10,1,9,12,2.4,2024-09-28,16:30,6,Sat,0.6,2.5,31413.0,Molineux Stadium,Anthony Taylor,19.0,52.5903,-2.1303,2024-09-28T16:30:00,11.878500,0,3,0,2024-09-28_Wolves_Liverpool,LIV,WOL,0.748925,2.117240,True,True,https://understat.com/match/26658,3.0,2.4807,1.356070,0.607145,14.777778,11.0,0.0,0.3471,0.748925,-0.607145,14.000000,2.0,57,6.0,5,2,Wolves,Liverpool,2024-09-28_Wolves_Liverpool,Wolves,Liverpool,8.00,5.75,1.33,3.0,6.0,16.0,8.0,2.0,10.0,2.0,3.0

# Feature Engineering

### Column names changed for feature engineering

In [79]:
import pandas as pd

data = table_df
# Mapping the old column names to new names where 'Home' and 'Away' are replaced with 'Team 1' and 'Team 2'
column_mapping = {
    'Home Team': 'Team 1',
    'Away Team': 'Team 2',
    'Home Score': 'Team 1 Score',
    'Away Score': 'Team 2 Score',
    'Prev Home Played': 'Prev Team 1 Played',
    'Prev Away Played': 'Prev Team 2 Played',
    'Prev Home Won': 'Prev Team 1 Won',
    'Prev Home Drawn': 'Prev Team 1 Drawn',
    'Prev Home Lost': 'Prev Team 1 Lost',
    'Prev Home Goals Scored': 'Prev Team 1 Goals Scored',
    'Prev Home Goals Conceded': 'Prev Team 1 Goals Conceded',
    'Prev Home Goal Difference': 'Prev Team 1 Goal Difference',
    'Prev Home Points': 'Prev Team 1 Points',
    'Prev Home Form': 'Prev Team 1 Form',
    'Prev Away Won': 'Prev Team 2 Won',
    'Prev Away Drawn': 'Prev Team 2 Drawn',
    'Prev Away Lost': 'Prev Team 2 Lost',
    'Prev Away Goals Scored': 'Prev Team 2 Goals Scored',
    'Prev Away Goals Conceded': 'Prev Team 2 Goals Conceded',
    'Prev Away Goal Difference': 'Prev Team 2 Goal Difference',
    'Prev Away Points': 'Prev Team 2 Points',
    'Prev Away Form': 'Prev Team 2 Form',
    'H2H_Home_Wins_Last_4': 'H2H_Team_1_Wins_Last_4',
    'H2H_Away_Wins_Last_4': 'H2H_Team_2_Wins_Last_4',
    'H2H_Draws_Last_4': 'H2H_Draws_Last_4',
    'home_team_code': 'team_1_code',
    'away_team_code': 'team_2_code',
    'home_xg': 'team_1_xg',
    'away_xg': 'team_2_xg',
    'home_team': 'team_1',
    'away_team': 'team_2',
    'HomeTeam': 'Team1',
    'AwayTeam': 'Team2',
    'home_team_fdr': 'team_1_fdr',
    'away_team_fdr': 'team_2_fdr',
    'home_deep_completions': 'team1_deep_completions',
    'away_deep_completions': 'team2_deep_completions',
    'away_np_xg': 'team2_np_xg',
    'home_np_xg': 'team1_np_xg',
    'away_np_xg_difference':'team2_np_xg_difference',
    'home_np_xg_difference':'home_np_xg_difference',
    'away_np_xg': 'team2_np_xg',
    'home_ppda':'team1_ppda', 
    'away_ppda': 'team2_ppda',
    'home_points': 'team1_points',
    'away_points': 'team2_points',
    'away_expected_points': 'team2_expected_points',
    'home_expected_points':'team1_expected_points',
    'home_np_xg_difference': 'team1_np_xg_difference',
    'Away_player_average': 'team2_player_average',
    'Home_player_average': 'team1_player_average',
    
}

# Renaming the columns in the dataframe
data.rename(columns=column_mapping, inplace=True)
data.columns


Index(['Match ID', 'Team 1', 'Team 2', 'Team 1 Score', 'Team 2 Score',
       'Winner', 'Prev Team 1 Played', 'Prev Team 1 Won', 'Prev Team 1 Drawn',
       'Prev Team 1 Lost', 'Prev Team 1 Goals Scored',
       'Prev Team 1 Goals Conceded', 'Prev Team 1 Goal Difference',
       'Prev Team 1 Points', 'Prev Team 1 Form', 'Prev Team 2 Played',
       'Prev Team 2 Won', 'Prev Team 2 Drawn', 'Prev Team 2 Lost',
       'Prev Team 2 Goals Scored', 'Prev Team 2 Goals Conceded',
       'Prev Team 2 Goal Difference', 'Prev Team 2 Points', 'Prev Team 2 Form',
       'Date', 'Time', 'Wk', 'Day', 'xG', 'xG.1', 'Attendance', 'Venue',
       'Referee', 'StadiumID', 'Latitude', 'Longitude', 'Datetime',
       'Temperature', 'H2H_Home_Total_Wins_Last_4',
       'H2H_Away_Total_Wins_Last_4', 'H2H_Draws_Last_4', 'Composite Key',
       'team_2_code', 'team_1_code', 'team_1_xg', 'team_2_xg', 'is_result',
       'has_data', 'url', 'team2_points', 'team2_expected_points',
       'team2_np_xg', 'team2_np_xg

In [80]:
data.tail(12)

,Match ID,Team 1,Team 2,Team 1 Score,Team 2 Score,Winner,Prev Team 1 Played,Prev Team 1 Won,Prev Team 1 Drawn,Prev Team 1 Lost,Prev Team 1 Goals Scored,Prev Team 1 Goals Conceded,Prev Team 1 Goal Difference,Prev Team 1 Points,Prev Team 1 Form,Prev Team 2 Played,Prev Team 2 Won,Prev Team 2 Drawn,Prev Team 2 Lost,Prev Team 2 Goals Scored,Prev Team 2 Goals Conceded,Prev Team 2 Goal Difference,Prev Team 2 Points,Prev Team 2 Form,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Home_Total_Wins_Last_4,H2H_Away_Total_Wins_Last_4,H2H_Draws_Last_4,Composite Key,team_2_code,team_1_code,team_1_xg,team_2_xg,is_result,has_data,url,team2_points,team2_expected_points,team2_np_xg,team2_np_xg_difference,team2_ppda,team2_deep_completions,team1_points,team1_expected_points,team1_np_xg,team1_np_xg_difference,team1_ppda,team1_deep_completions,Unnamed: 0_x,gameweek,team_1_fdr,team_2_fdr,team_1,team_2,composite_key,Team1,Team2,B365H,B365D,B365A,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,team1_player_average,team2_player_average,Unnamed: 0,Score,Away Win Odds,Home Win Odds,Draw Odds,Precipitation
0,497410,Manchester Utd,Fulham,1.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-16,19:00,1,Fri,2.4,0.4,73297.0,Old Trafford,Robert Jones,1.0,53.4631,-2.2914,2024-08-16T19:00:00,14.313000,1,0,0,2024-08-16_Manchester Utd_Fulham,FLH,MUN,2.042680,0.418711,True,True,https://understat.com/match/26602,0.0,0.2815,0.418711,-1.623969,10.833333,3.0,3.0,2.5696,2.042680,1.623969,7.379310,7.0,1,1.0,2,3,Manchester Utd,Fulham,2024-08-16_Manchester Utd_Fulham,Manchester Utd,Fulham,1.60,4.20,5.25,5.0,2.0,12.0,10.0,7.0,8.0,2.0,3.0,0.0,0.0,9.143500,8.974790,NaN,NaN,NaN,NaN,NaN,NaN
1,497412,Arsenal,Wolves,2.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00,1,Sat,1.2,0.5,60261.0,Emirates Stadium,Jarred Gillett,3.0,51.5550,-0.1086,2024-08-17T14:00:00,21.734999,3,0,0,2024-08-17_Arsenal_Wolves,WOL,ARS,1.628300,0.575835,True,True,https://understat.com/match/26604,0.0,0.5796,0.575835,-1.052465,10.818182,2.0,3.0,2.1894,1.628300,1.052465,7.769231,14.0,3,1.0,2,5,Arsenal,Wolves,2024-08-17_Arsenal_Wolves,Arsenal,Wolves,1.18,7.50,13.00,6.0,3.0,17.0,14.0,8.0,2.0,2.0,2.0,0.0,0.0,10.830699,6.027272,NaN,NaN,NaN,NaN,NaN,NaN
2,497436,Manchester Utd,Liverpool,0.0,3.0,AWAY_TEAM,2,1,0,1,2,2,0,3,1.5,2,2,0,0,4,0,4,6,3.0,2024-09-01,15:00,3,Sun,1.4,1.8,73738.0,Old Trafford,Anthony Taylor,1.0,53.4631,-2.2914,2024-09-01T15:00:00,19.213001,1,2,0,2024-09-01_Manchester Utd_Liverpool,LIV,MUN,1.500870,2.103970,True,True,https://understat.com/match/26631,3.0,1.7849,2.103970,0.603100,11.692308,9.0,0.0,0.9689,1.500870,-0.603100,12.500000,6.0,30,3.0,5,3,Manchester Utd,Liverpool,2024-09-01_Manchester Utd_Liverpool,Manchester Utd,Liverpool,3.70,4.20,1.85,3.0,3.0,7.0,7.0,5.0,2.0,4.0,1.0,0.0,0.0,4.950962,9.905478,NaN,NaN,NaN,NaN,NaN,NaN
3,497447,Southampton,Manchester Utd,0.0,3.0,AWAY_TEAM,3,0,0,3,1,5,-4,0,0.0,3,1,0,2,2,5,-3,3,1.0,2024-09-14,11:30,4,Sat,1.1,2.6,31144.0,St Mary's Stadium,Stuart Attwell,15.0,50.9058,-1.3913,2024-09-14T11:30:00,16.136499,0,1,0,2024-09-14_Southampton_Manchester Utd,MUN,SOU,1.355930,3.026930,True,True,https://understat.com/match/26632,3.0,2.5032,3.026930,2.481613,10.344828,17.0,0.0,0.3435,0.545317,-2.481613,10.857143,3.0,31,4.0,3,2,Southampton,Manchester Utd,2024-09-14_Southampton_Manchester Utd,Southampton,Manchester Utd,4.50,4.00,1.70,4.0,10.0,11.0,14.0,0.0,7.0,1.0,4.0,1.0,0.0,8.010871,11.243967,NaN,NaN,NaN,NaN,NaN,NaN
4,497469,Wolves,Liverpool,1.0,2.0,AWAY_TEAM,5,0,1,4,5,14,-9,1,0.2,5,4,0,1,10,1,9,12,2.4,2024-09-28,16:30,6,Sat,0.6,2.5,31413.0,Molineux Stadium,Anthony Taylor,19.0,52.5903,-2.1303,2024-09-28T16:30:00,11.878500,0,3,0,2024-09-28_Wolves_Liverpool,LIV,WOL,0.748925,2.117240,True,True,https://understat.com/match/26658,3.0,2.4807,1.356070,0.607145,14.777778,11.0,0.0,0.3471,0.748925,-0.607145,14.000000,2.0,57,6.0,5,2,Wolves,Liverpool,2024-09-28_Wolves_Liverpool,Wolves,Liverpool,8.00,5.75,1.33,3.0,6

### FE: Average fdr

In [82]:
def calculate_fdr_cumulative_features(dataframe):
    # Initialize dictionaries to track cumulative sums and games played
    cumulative_fdr = {}
    games_played = {}
    
    # Create new columns for average FDR and cumulative FDR
    avg_team_1_fdr = []
    avg_team_2_fdr = []
    cum_team_1_fdr = []
    cum_team_2_fdr = []

    # Iterate over rows
    for index, row in dataframe.iterrows():
        team_1 = row["Team 1"]
        team_2 = row["Team 2"]
        
        # Initialize if not present in dictionary
        if team_1 not in cumulative_fdr:
            cumulative_fdr[team_1] = 0
            games_played[team_1] = 0
        if team_2 not in cumulative_fdr:
            cumulative_fdr[team_2] = 0
            games_played[team_2] = 0

        # Update cumulative FDR
        cumulative_fdr[team_1] += row['team_1_fdr']
        cumulative_fdr[team_2] += row['team_2_fdr']
        
        # Increment games played
        games_played[team_1] += 1
        games_played[team_2] += 1

         # Record cumulative FDR for the current row
        cum_team_1_fdr.append(cumulative_fdr[team_1])
        cum_team_2_fdr.append(cumulative_fdr[team_2])
        
        # Calculate averages for teams before updating current match FDR
       
        avg_team_1_fdr.append(cumulative_fdr[team_1] / (games_played[team_1])) 
        avg_team_2_fdr.append(cumulative_fdr[team_2] / (games_played[team_2]))

    # Add new columns to the DataFrame
    dataframe['Cum Team 1 FDR'] = cum_team_1_fdr
    dataframe['Cum Team 2 FDR'] = cum_team_2_fdr
    dataframe['Avg Team 1 FDR'] = avg_team_1_fdr
    dataframe['Avg Team 2 FDR'] = avg_team_2_fdr

    

    return dataframe

# Calculate cumulative features excluding current match
detailed_fixtures = calculate_fdr_cumulative_features(data)

# Display selected rows with cumulative values and averages
columns_to_display = [
    'Match ID', 'Team 1', 'Team 2', 'team_1_fdr', 'team_2_fdr',
    'Cum Team 1 FDR', 'Cum Team 2 FDR', 'Avg Team 1 FDR', 'Avg Team 2 FDR'
]



In [83]:
detailed_fixtures[['Team 1','Team 2','Prev Team 1 Played',
'Prev Team 2 Played','Avg Team 1 FDR','Avg Team 2 FDR','team_1_fdr', 'team_2_fdr','Cum Team 1 FDR','Cum Team 2 FDR']].to_csv("detailed_fixture.csv")

In [84]:
table_df.head()

,Match ID,Team 1,Team 2,Team 1 Score,Team 2 Score,Winner,Prev Team 1 Played,Prev Team 1 Won,Prev Team 1 Drawn,Prev Team 1 Lost,Prev Team 1 Goals Scored,Prev Team 1 Goals Conceded,Prev Team 1 Goal Difference,Prev Team 1 Points,Prev Team 1 Form,Prev Team 2 Played,Prev Team 2 Won,Prev Team 2 Drawn,Prev Team 2 Lost,Prev Team 2 Goals Scored,Prev Team 2 Goals Conceded,Prev Team 2 Goal Difference,Prev Team 2 Points,Prev Team 2 Form,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Home_Total_Wins_Last_4,H2H_Away_Total_Wins_Last_4,H2H_Draws_Last_4,Composite Key,team_2_code,team_1_code,team_1_xg,team_2_xg,is_result,has_data,url,team2_points,team2_expected_points,team2_np_xg,team2_np_xg_difference,team2_ppda,team2_deep_completions,team1_points,team1_expected_points,team1_np_xg,team1_np_xg_difference,team1_ppda,team1_deep_completions,Unnamed: 0_x,gameweek,team_1_fdr,team_2_fdr,team_1,team_2,composite_key,Team1,Team2,B365H,B365D,B365A,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,team1_player_average,team2_player_average,Unnamed: 0,Score,Away Win Odds,Home Win Odds,Draw Odds,Precipitation,Cum Team 1 FDR,Cum Team 2 FDR,Avg Team 1 FDR,Avg Team 2 FDR
0,497410,Manchester Utd,Fulham,1.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-16,19:00,1,Fri,2.4,0.4,73297.0,Old Trafford,Robert Jones,1.0,53.4631,-2.2914,2024-08-16T19:00:00,14.313000,1,0,0,2024-08-16_Manchester Utd_Fulham,FLH,MUN,2.042680,0.418711,True,True,https://understat.com/match/26602,0.0,0.2815,0.418711,-1.623969,10.833333,3.0,3.0,2.5696,2.042680,1.623969,7.379310,7.0,1,1.0,2,3,Manchester Utd,Fulham,2024-08-16_Manchester Utd_Fulham,Manchester Utd,Fulham,1.60,4.20,5.25,5.0,2.0,12.0,10.0,7.0,8.0,2.0,3.0,0.0,0.0,9.143500,8.974790,NaN,NaN,NaN,NaN,NaN,NaN,2,3,2.0,3.0
1,497412,Arsenal,Wolves,2.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00,1,Sat,1.2,0.5,60261.0,Emirates Stadium,Jarred Gillett,3.0,51.5550,-0.1086,2024-08-17T14:00:00,21.734999,3,0,0,2024-08-17_Arsenal_Wolves,WOL,ARS,1.628300,0.575835,True,True,https://understat.com/match/26604,0.0,0.5796,0.575835,-1.052465,10.818182,2.0,3.0,2.1894,1.628300,1.052465,7.769231,14.0,3,1.0,2,5,Arsenal,Wolves,2024-08-17_Arsenal_Wolves,Arsenal,Wolves,1.18,7.50,13.00,6.0,3.0,17.0,14.0,8.0,2.0,2.0,2.0,0.0,0.0,10.830699,6.027272,NaN,NaN,NaN,NaN,NaN,NaN,2,5,2.0,5.0
2,497436,Manchester Utd,Liverpool,0.0,3.0,AWAY_TEAM,2,1,0,1,2,2,0,3,1.5,2,2,0,0,4,0,4,6,3.0,2024-09-01,15:00,3,Sun,1.4,1.8,73738.0,Old Trafford,Anthony Taylor,1.0,53.4631,-2.2914,2024-09-01T15:00:00,19.213001,1,2,0,2024-09-01_Manchester Utd_Liverpool,LIV,MUN,1.500870,2.103970,True,True,https://understat.com/match/26631,3.0,1.7849,2.103970,0.603100,11.692308,9.0,0.0,0.9689,1.500870,-0.603100,12.500000,6.0,30,3.0,5,3,Manchester Utd,Liverpool,2024-09-01_Manchester Utd_Liverpool,Manchester Utd,Liverpool,3.70,4.20,1.85,3.0,3.0,7.0,7.0,5.0,2.0,4.0,1.0,0.0,0.0,4.950962,9.905478,NaN,NaN,NaN,NaN,NaN,NaN,7,3,3.5,3.0
3,497447,Southampton,Manchester Utd,0.0,3.0,AWAY_TEAM,3,0,0,3,1,5,-4,0,0.0,3,1,0,2,2,5,-3,3,1.0,2024-09-14,11:30,4,Sat,1.1,2.6,31144.0,St Mary's Stadium,Stuart Attwell,15.0,50.9058,-1.3913,2024-09-14T11:30:00,16.136499,0,1,0,2024-09-14_Southampton_Manchester Utd,MUN,SOU,1.355930,3.026930,True,True,https://understat.com/match/26632,3.0,2.5032,3.026930,2.481613,10.344828,17.0,0.0,0.3435,0.545317,-2.481613,10.857143,3.0,31,4.0,3,2,Southampton,Manchester Utd,2024-09-14_Southampton_Manchester Utd,Southampton,Manchester Utd,4.50,4.00,1.70,4.0,10.0,11.0,14.0,0.0,7.0,1.0,4.0,1.0,0.0,8.010871,11.243967,NaN,NaN,NaN,NaN,NaN,NaN,3,9,3.0,3.0
4,497469,Wolves,Liverpool,1.0,2.0,AWAY_TEAM,5,0,1,4,5,14,-9,1,0.2,5,4,0,1,10,1,9,12,2.4,2024-09-28,16:30,6,Sat,0.6,2.5,31413.0,Molineux Stadium,Anthony Taylor,19.0,52.5903,-2.1303,2024-09-28T16:30:00,11.878500,0,3,0,2024-09-28_Wolves_Liverpool,LIV,WOL,0.748925,2.117240,True,True,https://understat.com/match/26658,3.0,2.4807,1.356070,0.607145,14.777778,11.0,0.0,0.3471,0.748925,-0.607145

In [85]:
table_df = table_df.drop(columns=['team_1', 'team_2','Team1', 'Team2'])

In [86]:
table_df.to_csv("fact_table.csv")

In [87]:
df = table_df

In [88]:
df.columns

Index(['Match ID', 'Team 1', 'Team 2', 'Team 1 Score', 'Team 2 Score',
       'Winner', 'Prev Team 1 Played', 'Prev Team 1 Won', 'Prev Team 1 Drawn',
       'Prev Team 1 Lost', 'Prev Team 1 Goals Scored',
       'Prev Team 1 Goals Conceded', 'Prev Team 1 Goal Difference',
       'Prev Team 1 Points', 'Prev Team 1 Form', 'Prev Team 2 Played',
       'Prev Team 2 Won', 'Prev Team 2 Drawn', 'Prev Team 2 Lost',
       'Prev Team 2 Goals Scored', 'Prev Team 2 Goals Conceded',
       'Prev Team 2 Goal Difference', 'Prev Team 2 Points', 'Prev Team 2 Form',
       'Date', 'Time', 'Wk', 'Day', 'xG', 'xG.1', 'Attendance', 'Venue',
       'Referee', 'StadiumID', 'Latitude', 'Longitude', 'Datetime',
       'Temperature', 'H2H_Home_Total_Wins_Last_4',
       'H2H_Away_Total_Wins_Last_4', 'H2H_Draws_Last_4', 'Composite Key',
       'team_2_code', 'team_1_code', 'team_1_xg', 'team_2_xg', 'is_result',
       'has_data', 'url', 'team2_points', 'team2_expected_points',
       'team2_np_xg', 'team2_np_xg

### FE: Average ppda/deep completion

In [90]:
def calculate_weighted_ppda_deep_completions(df):
    # Initialize dictionaries to track cumulative sums and games played
    cumulative_ppda = {}
    cumulative_deep = {}
    cumulative_fdr = {}
    games_played = {}

    # Create new columns for weighted average PPDA and deep completions
    cum_ppda_team1 = []
    cum_ppda_team2 = []
    cum_deep_team1 = []
    cum_deep_team2 = []
    avg_ppda_team1 = []
    avg_ppda_team2 = []
    avg_deep_team1 = []
    avg_deep_team2 = []
    weighted_avg_ppda_team1 = []
    weighted_avg_ppda_team2 = []
    weighted_avg_deep_team1 = []
    weighted_avg_deep_team2 = []

    # Iterate over rows
    for index, row in df.iterrows():
        team1 = row["Team 1"]
        team2 = row["Team 2"]

        # Initialize if not present in the dictionary
        for team in (team1, team2):
            if team not in cumulative_ppda:
                cumulative_ppda[team] = 0
                cumulative_deep[team] = 0
                cumulative_fdr[team] = 0
                games_played[team] = 0

        # Record cumulative values for the current row before updating them
        cum_ppda_team1.append(cumulative_ppda[team1])
        cum_ppda_team2.append(cumulative_ppda[team2])
        cum_deep_team1.append(cumulative_deep[team1])
        cum_deep_team2.append(cumulative_deep[team2])

        # Calculate averages based on games played before this match
        if games_played[team1] > 0:
            avg_ppda_team1.append(cumulative_ppda[team1] / games_played[team1])
            avg_deep_team1.append(cumulative_deep[team1] / games_played[team1])
            weighted_avg_ppda_team1.append((cumulative_ppda[team1] / games_played[team1]) * (cumulative_fdr[team1] / games_played[team1]))
            weighted_avg_deep_team1.append((cumulative_deep[team1] / games_played[team1]) * (cumulative_fdr[team1] / games_played[team1]))
        else:
            avg_ppda_team1.append(0)
            avg_deep_team1.append(0)
            weighted_avg_ppda_team1.append(0)
            weighted_avg_deep_team1.append(0)

        if games_played[team2] > 0:
            avg_ppda_team2.append(cumulative_ppda[team2] / games_played[team2])
            avg_deep_team2.append(cumulative_deep[team2] / games_played[team2])
            weighted_avg_ppda_team2.append((cumulative_ppda[team2] / games_played[team2]) * (cumulative_fdr[team2] / games_played[team2]))
            weighted_avg_deep_team2.append((cumulative_deep[team2] / games_played[team2]) * (cumulative_fdr[team2] / games_played[team2]))
        else:
            avg_ppda_team2.append(0)
            avg_deep_team2.append(0)
            weighted_avg_ppda_team2.append(0)
            weighted_avg_deep_team2.append(0)

        # Update cumulative PPDA, deep completions, and FDR after recording the current match
        cumulative_ppda[team1] += row['team1_ppda']
        cumulative_ppda[team2] += row['team2_ppda']
        cumulative_deep[team1] += row['team1_deep_completions']
        cumulative_deep[team2] += row['team2_deep_completions']
        cumulative_fdr[team1] += row['team_1_fdr']
        cumulative_fdr[team2] += row['team_2_fdr']

        # Increment games played after using the counts
        games_played[team1] += 1
        games_played[team2] += 1

    # Add new columns to the DataFrame
    df['Cum PPDA Team 1'] = cum_ppda_team1
    df['Cum PPDA Team 2'] = cum_ppda_team2
    df['Avg PPDA Team 1'] = avg_ppda_team1
    df['Avg PPDA Team 2'] = avg_ppda_team2
    df['Cum Deep Completions Team 1'] = cum_deep_team1
    df['Cum Deep Completions Team 2'] = cum_deep_team2
    df['Avg Deep Completions Team 1'] = avg_deep_team1
    df['Avg Deep Completions Team 2'] = avg_deep_team2
    df['Weighted Avg PPDA Team 1'] = weighted_avg_ppda_team1
    df['Weighted Avg PPDA Team 2'] = weighted_avg_ppda_team2
    df['Weighted Avg Deep Completions Team 1'] = weighted_avg_deep_team1
    df['Weighted Avg Deep Completions Team 2'] = weighted_avg_deep_team2

    return df

# Example usage with your dataset
updated_data = calculate_weighted_ppda_deep_completions(df)

# Display selected rows with weighted cumulative values and averages
columns_to_display = [
    'Match ID', 'Team 1', 'Team 2', 'team1_ppda', 'team2_ppda',
    'Weighted Avg PPDA Team 1', 'Weighted Avg PPDA Team 2',
    'Weighted Avg Deep Completions Team 1', 'Weighted Avg Deep Completions Team 2'
]
display_df = updated_data[columns_to_display]


In [91]:
display_df.head(12)

,Match ID,Team 1,Team 2,team1_ppda,team2_ppda,Weighted Avg PPDA Team 1,Weighted Avg PPDA Team 2,Weighted Avg Deep Completions Team 1,Weighted Avg Deep Completions Team 2
0,497410,Manchester Utd,Fulham,7.379310,10.833333,0.000000,0.000000,0.000,0.00
1,497412,Arsenal,Wolves,7.769231,10.818182,0.000000,0.000000,0.000,0.00
2,497436,Manchester Utd,Liverpool,12.500000,11.692308,14.758621,0.000000,14.000,0.00
3,497447,Southampton,Manchester Utd,10.857143,10.344828,0.000000,34.788793,0.000,22.75
4,497469,Wolves,Liverpool,14.000000,14.777778,54.090909,35.076923,10.000,27.00
5,497519,Wolves,Southampton,25.437500,5.814815,62.045455,32.571429,10.000,9.00
6,497541,Arsenal,Manchester Utd,11.800000,19.400000,15.538462,30.224138,28.000,30.00
7,497577,Manchester Utd,Bournemouth,NaN,NaN,43.421121,0.000000,27.125,0.00


In [92]:
updated_data.head(12)

,Match ID,Team 1,Team 2,Team 1 Score,Team 2 Score,Winner,Prev Team 1 Played,Prev Team 1 Won,Prev Team 1 Drawn,Prev Team 1 Lost,Prev Team 1 Goals Scored,Prev Team 1 Goals Conceded,Prev Team 1 Goal Difference,Prev Team 1 Points,Prev Team 1 Form,Prev Team 2 Played,Prev Team 2 Won,Prev Team 2 Drawn,Prev Team 2 Lost,Prev Team 2 Goals Scored,Prev Team 2 Goals Conceded,Prev Team 2 Goal Difference,Prev Team 2 Points,Prev Team 2 Form,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Home_Total_Wins_Last_4,H2H_Away_Total_Wins_Last_4,H2H_Draws_Last_4,Composite Key,team_2_code,team_1_code,team_1_xg,team_2_xg,is_result,has_data,url,team2_points,team2_expected_points,team2_np_xg,team2_np_xg_difference,team2_ppda,team2_deep_completions,team1_points,team1_expected_points,team1_np_xg,team1_np_xg_difference,team1_ppda,team1_deep_completions,Unnamed: 0_x,gameweek,team_1_fdr,team_2_fdr,composite_key,B365H,B365D,B365A,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,team1_player_average,team2_player_average,Unnamed: 0,Score,Away Win Odds,Home Win Odds,Draw Odds,Precipitation,Cum Team 1 FDR,Cum Team 2 FDR,Avg Team 1 FDR,Avg Team 2 FDR,Cum PPDA Team 1,Cum PPDA Team 2,Avg PPDA Team 1,Avg PPDA Team 2,Cum Deep Completions Team 1,Cum Deep Completions Team 2,Avg Deep Completions Team 1,Avg Deep Completions Team 2,Weighted Avg PPDA Team 1,Weighted Avg PPDA Team 2,Weighted Avg Deep Completions Team 1,Weighted Avg Deep Completions Team 2
0,497410,Manchester Utd,Fulham,1.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-16,19:00,1,Fri,2.4,0.4,73297.0,Old Trafford,Robert Jones,1.0,53.4631,-2.2914,2024-08-16T19:00:00,14.313000,1,0,0,2024-08-16_Manchester Utd_Fulham,FLH,MUN,2.042680,0.418711,True,True,https://understat.com/match/26602,0.0,0.2815,0.418711,-1.623969,10.833333,3.0,3.0,2.5696,2.042680,1.623969,7.379310,7.0,1,1.0,2,3,2024-08-16_Manchester Utd_Fulham,1.60,4.20,5.25,5.0,2.0,12.0,10.0,7.0,8.0,2.0,3.0,0.0,0.0,9.143500,8.974790,NaN,NaN,NaN,NaN,NaN,NaN,2,3,2.0,3.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.0,0.000000,0.000000,0.000,0.00
1,497412,Arsenal,Wolves,2.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00,1,Sat,1.2,0.5,60261.0,Emirates Stadium,Jarred Gillett,3.0,51.5550,-0.1086,2024-08-17T14:00:00,21.734999,3,0,0,2024-08-17_Arsenal_Wolves,WOL,ARS,1.628300,0.575835,True,True,https://understat.com/match/26604,0.0,0.5796,0.575835,-1.052465,10.818182,2.0,3.0,2.1894,1.628300,1.052465,7.769231,14.0,3,1.0,2,5,2024-08-17_Arsenal_Wolves,1.18,7.50,13.00,6.0,3.0,17.0,14.0,8.0,2.0,2.0,2.0,0.0,0.0,10.830699,6.027272,NaN,NaN,NaN,NaN,NaN,NaN,2,5,2.0,5.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.0,0.000000,0.000000,0.000,0.00
2,497436,Manchester Utd,Liverpool,0.0,3.0,AWAY_TEAM,2,1,0,1,2,2,0,3,1.5,2,2,0,0,4,0,4,6,3.0,2024-09-01,15:00,3,Sun,1.4,1.8,73738.0,Old Trafford,Anthony Taylor,1.0,53.4631,-2.2914,2024-09-01T15:00:00,19.213001,1,2,0,2024-09-01_Manchester Utd_Liverpool,LIV,MUN,1.500870,2.103970,True,True,https://understat.com/match/26631,3.0,1.7849,2.103970,0.603100,11.692308,9.0,0.0,0.9689,1.500870,-0.603100,12.500000,6.0,30,3.0,5,3,2024-09-01_Manchester Utd_Liverpool,3.70,4.20,1.85,3.0,3.0,7.0,7.0,5.0,2.0,4.0,1.0,0.0,0.0,4.950962,9.905478,NaN,NaN,NaN,NaN,NaN,NaN,7,3,3.5,3.0,7.379310,0.000000,7.379310,0.000000,7.0,0.0,7.00,0.0,14.758621,0.000000,14.000,0.00
3,497447,Southampton,Manchester Utd,0.0,3.0,AWAY_TEAM,3,0,0,3,1,5,-4,0,0.0,3,1,0,2,2,5,-3,3,1.0,2024-09-14,11:30,4,Sat,1.1,2.6,31144.0,St Mary's Stadium,Stuart Attwell,15.0,50.9058,-1.3913,2024-09-14T11:30:00,16.136499,0,1,0,2024-09-14_Southampton_Manchester Utd,MUN,SOU,1.355930,3.026930,True,True,https://understat.com/match/26632,3.0,2.5032,3.026930,2.481613,10.344828,17.0,0.0,0.3435,0.545317,-2.481613,10.857143,3.0,31,4.0,3,2,2024-09-14_Southampton_Manchester Utd,4.50,4.00,1.70,4.0,10.0,11.0,14.0,0.0,7.0,1.0,4.0,1.0,0.0,8.010871,11.243967,NaN,NaN,NaN,NaN,NaN,NaN,3,9,3.0,3.0,0.000000,19.879310,0.000000,9.939655,0.0,13

In [93]:
updated_data.columns

Index(['Match ID', 'Team 1', 'Team 2', 'Team 1 Score', 'Team 2 Score',
       'Winner', 'Prev Team 1 Played', 'Prev Team 1 Won', 'Prev Team 1 Drawn',
       'Prev Team 1 Lost',
       ...
       'Avg PPDA Team 1', 'Avg PPDA Team 2', 'Cum Deep Completions Team 1',
       'Cum Deep Completions Team 2', 'Avg Deep Completions Team 1',
       'Avg Deep Completions Team 2', 'Weighted Avg PPDA Team 1',
       'Weighted Avg PPDA Team 2', 'Weighted Avg Deep Completions Team 1',
       'Weighted Avg Deep Completions Team 2'],
      dtype='object', length=103)

### FE: Avg xg stats

In [95]:
df.tail()

,Match ID,Team 1,Team 2,Team 1 Score,Team 2 Score,Winner,Prev Team 1 Played,Prev Team 1 Won,Prev Team 1 Drawn,Prev Team 1 Lost,Prev Team 1 Goals Scored,Prev Team 1 Goals Conceded,Prev Team 1 Goal Difference,Prev Team 1 Points,Prev Team 1 Form,Prev Team 2 Played,Prev Team 2 Won,Prev Team 2 Drawn,Prev Team 2 Lost,Prev Team 2 Goals Scored,Prev Team 2 Goals Conceded,Prev Team 2 Goal Difference,Prev Team 2 Points,Prev Team 2 Form,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Home_Total_Wins_Last_4,H2H_Away_Total_Wins_Last_4,H2H_Draws_Last_4,Composite Key,team_2_code,team_1_code,team_1_xg,team_2_xg,is_result,has_data,url,team2_points,team2_expected_points,team2_np_xg,team2_np_xg_difference,team2_ppda,team2_deep_completions,team1_points,team1_expected_points,team1_np_xg,team1_np_xg_difference,team1_ppda,team1_deep_completions,Unnamed: 0_x,gameweek,team_1_fdr,team_2_fdr,composite_key,B365H,B365D,B365A,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,team1_player_average,team2_player_average,Unnamed: 0,Score,Away Win Odds,Home Win Odds,Draw Odds,Precipitation,Cum Team 1 FDR,Cum Team 2 FDR,Avg Team 1 FDR,Avg Team 2 FDR,Cum PPDA Team 1,Cum PPDA Team 2,Avg PPDA Team 1,Avg PPDA Team 2,Cum Deep Completions Team 1,Cum Deep Completions Team 2,Avg Deep Completions Team 1,Avg Deep Completions Team 2,Weighted Avg PPDA Team 1,Weighted Avg PPDA Team 2,Weighted Avg Deep Completions Team 1,Weighted Avg Deep Completions Team 2
3,497447,Southampton,Manchester Utd,0.0,3.0,AWAY_TEAM,3,0,0,3,1,5,-4,0,0.0,3,1,0,2,2,5,-3,3,1.0,2024-09-14,11:30,4,Sat,1.1,2.6,31144.0,St Mary's Stadium,Stuart Attwell,15.0,50.9058,-1.3913,2024-09-14T11:30:00,16.136499,0,1,0,2024-09-14_Southampton_Manchester Utd,MUN,SOU,1.355930,3.026930,True,True,https://understat.com/match/26632,3.0,2.5032,3.026930,2.481613,10.344828,17.0,0.0,0.3435,0.545317,-2.481613,10.857143,3.0,31,4.0,3,2,2024-09-14_Southampton_Manchester Utd,4.50,4.00,1.70,4.0,10.0,11.0,14.0,0.0,7.0,1.0,4.0,1.0,0.0,8.010871,11.243967,NaN,NaN,NaN,NaN,NaN,NaN,3,9,3.0,3.0,0.000000,19.879310,0.000000,9.939655,0.0,13.0,0.00,6.5,0.000000,34.788793,0.000,22.75
4,497469,Wolves,Liverpool,1.0,2.0,AWAY_TEAM,5,0,1,4,5,14,-9,1,0.2,5,4,0,1,10,1,9,12,2.4,2024-09-28,16:30,6,Sat,0.6,2.5,31413.0,Molineux Stadium,Anthony Taylor,19.0,52.5903,-2.1303,2024-09-28T16:30:00,11.878500,0,3,0,2024-09-28_Wolves_Liverpool,LIV,WOL,0.748925,2.117240,True,True,https://understat.com/match/26658,3.0,2.4807,1.356070,0.607145,14.777778,11.0,0.0,0.3471,0.748925,-0.607145,14.000000,2.0,57,6.0,5,2,2024-09-28_Wolves_Liverpool,8.00,5.75,1.33,3.0,6.0,16.0,8.0,2.0,10.0,2.0,3.0,0.0,0.0,5.327128,11.979388,NaN,NaN,NaN,NaN,NaN,NaN,10,5,5.0,2.5,10.818182,11.692308,10.818182,11.692308,2.0,9.0,2.00,9.0,54.090909,35.076923,10.000,27.00
5,497519,Wolves,Southampton,2.0,0.0,HOME_TEAM,10,0,3,7,14,27,-13,3,0.4,10,1,1,8,7,19,-12,4,0.6,2024-11-09,15:00,11,Sat,1.3,0.6,31403.0,Molineux Stadium,Thomas Bramall,19.0,52.5903,-2.1303,2024-11-09T15:00:00,7.128500,0,0,0,2024-11-09_Wolves_Southampton,SOU,WOL,0.912781,0.792655,True,True,https://understat.com/match/26711,0.0,1.1777,0.792655,-0.120126,5.814815,6.0,3.0,1.4630,0.912781,0.120126,25.437500,7.0,104,11.0,2,2,2024-11-09_Wolves_Southampton,1.91,3.75,3.75,4.0,0.0,14.0,12.0,1.0,9.0,1.0,4.0,0.0,0.0,12.271554,9.996454,NaN,NaN,NaN,NaN,NaN,NaN,12,5,4.0,2.5,24.818182,10.857143,12.409091,10.857143,4.0,3.0,2.00,3.0,62.045455,32.571429,10.000,9.00
6,497541,Arsenal,Manchester Utd,2.0,0.0,HOME_TEAM,13,7,4,2,26,14,12,25,1.6,13,5,4,4,17,13,4,19,1.6,2024-12-04,20:15,14,Wed,2.1,0.2,60256.0,Emirates Stadium,Samuel Barrott,3.0,51.5550,-0.1086,2024-12-04T20:15:00,15.067417,1,2,0,2024-12-04_Arsenal_Manchester Utd,MUN,ARS,2.966840,0.256083,True,True,https://understat.com/match/26733,0.0,0.0343,0.256083,-2.710757,19.400000,1.0,3.0,2.9395,2.966840,2.710757,11.800000,8.0,137,14.0,3,5,2024-12-04_Arsenal_Manchester Utd,1.44,4.50,7.00,6.0,2.0,12.0,8.0,13.0,0.0,1.0,3.0,0.0,0.0,8.972388,5.079682,NaN,NaN,NaN,NaN,NaN,NaN,5

In [96]:
def calculate_weighted_xg_stats(df):
    # Initialize dictionaries to track cumulative sums and games played
    xg_stats = [
        'xG', 'xG.1',
    ]
    cumulative = {stat: {} for stat in xg_stats}
    games_played = {}

    # Initialize columns for cumulative, average, and weighted average stats
    for stat in xg_stats:
        df[f'Cum {stat}'] = 0
        df[f'Avg {stat}'] = 0.0
        df[f'Weighted Avg {stat}'] = 0.0

    # Iterate over rows
    for index, row in df.iterrows():
        team1 = row["Team 1"]
        team2 = row["Team 2"]

        # Initialize if not present in the dictionary
        for team in [team1, team2]:
            if team not in games_played:
                games_played[team] = 0
                for stat in xg_stats:
                    cumulative[stat][team] = 0

        # Compute averages for each stat based on previous matches' data
        for stat in xg_stats:
            team_key = team1 if '.1' not in stat and 'team2' not in stat else team2
            corrected_stat = stat.replace('.1', '') if '.1' in stat else stat  # Adjust the stat key for team2

            # Before updating the cumulative, record the current game's value
            value_to_add = row[stat] if pd.notna(row[stat]) else 0

            # Calculate averages using prior match data
            if games_played[team_key] >= 1:  # Now checks if there's at least one past game
                current_avg = cumulative[corrected_stat][team_key] / games_played[team_key]
                df.at[index, f'Avg {stat}'] = current_avg
                fdr = row['Avg Team 1 FDR'] if team_key == team1 else row['Avg Team 2 FDR']
                df.at[index, f'Weighted Avg {stat}'] = current_avg * fdr
            else:
                df.at[index, f'Avg {stat}'] = 0
                df.at[index, f'Weighted Avg {stat}'] = 0

            # Update cumulative values after computing averages
            cumulative[corrected_stat][team_key] += value_to_add

        # Increment games played after updating stats
        games_played[team1] += 1
        games_played[team2] += 1

    return df

In [97]:
# Example usage with your dataset
data_df = calculate_weighted_xg_stats(df)

# Display selected rows with weighted cumulative values and averages
columns_to_display = [
    'Match ID', 'Team 1', 'Team 2', 'xG', 'xG.1','Avg xG','Avg xG.1', 'Weighted Avg xG', 'Weighted Avg xG.1'
]
display_df = data_df[columns_to_display]


In [98]:
display_df.tail()

,Match ID,Team 1,Team 2,xG,xG.1,Avg xG,Avg xG.1,Weighted Avg xG,Weighted Avg xG.1
3,497447,Southampton,Manchester Utd,1.1,2.6,0.00,1.900000,0.00,5.700000
4,497469,Wolves,Liverpool,0.6,2.5,0.50,1.800000,2.50,4.500000
5,497519,Wolves,Southampton,1.3,0.6,0.55,1.100000,2.20,2.750000
6,497541,Arsenal,Manchester Utd,2.1,0.2,1.20,2.133333,3.00,7.466667
7,497577,Manchester Utd,Bournemouth,NaN,NaN,1.65,0.000000,5.28,0.000000


In [99]:
def calculate_weighted_xg_stats(df):
    # Initialize dictionaries to track cumulative sums and games played
    cumulative_xg = {}
    games_played = {}

    # Create new columns for xG stats
    cum_xg_team1 = []
    cum_xg_team2 = []
    avg_xg_team1 = []
    avg_xg_team2 = []
    weighted_avg_xg_team1 = []
    weighted_avg_xg_team2 = []

    # Iterate over rows
    for index, row in df.iterrows():
        team1 = row["Team 1"]
        team2 = row["Team 2"]

        # Initialize if not present in the dictionary
        for team in (team1, team2):
            if team not in cumulative_xg:
                cumulative_xg[team] = 0
                games_played[team] = 0

        # Record cumulative values for the current row before updating them
        cum_xg_team1.append(cumulative_xg[team1])
        cum_xg_team2.append(cumulative_xg[team2])

        # Calculate averages based on games played before this match
        if games_played[team1] > 0:
            avg_xg_team1.append(cumulative_xg[team1] / games_played[team1])
            # Multiply the average by 'Avg Team 1 FDR' for weighted average
            weighted_avg_xg_team1.append(avg_xg_team1[-1] * row['Avg Team 1 FDR'])
        else:
            avg_xg_team1.append(0)
            weighted_avg_xg_team1.append(0)

        if games_played[team2] > 0:
            avg_xg_team2.append(cumulative_xg[team2] / games_played[team2])
            # Multiply the average by 'Avg Team 2 FDR' for weighted average
            weighted_avg_xg_team2.append(avg_xg_team2[-1] * row['Avg Team 2 FDR'])
        else:
            avg_xg_team2.append(0)
            weighted_avg_xg_team2.append(0)

        # Update cumulative xG after recording the current match
        cumulative_xg[team1] += row['team_1_xg']
        cumulative_xg[team2] += row['team_2_xg']

        # Increment games played after using the counts
        games_played[team1] += 1
        games_played[team2] += 1

    # Add new columns to the DataFrame
    df['Cum xG Team 1'] = cum_xg_team1
    df['Cum xG Team 2'] = cum_xg_team2
    df['Avg xG Team 1'] = avg_xg_team1
    df['Avg xG Team 2'] = avg_xg_team2
    df['Weighted Avg xG Team 1'] = weighted_avg_xg_team1
    df['Weighted Avg xG Team 2'] = weighted_avg_xg_team2

    return df
    
# Example usage with your dataset
updated_data = calculate_weighted_xg_stats(data_df)

# Display selected rows with weighted cumulative values and averages
columns_to_display = [
    'Match ID', 'Team 1', 'Team 2', 'team_1_xg', 'team_2_xg',
    'Avg xG Team 1', 'Avg xG Team 2',
    'Weighted Avg xG Team 1', 'Weighted Avg xG Team 2',  'xG', 'xG.1','Avg xG','Avg xG.1', 'Weighted Avg xG', 'Weighted Avg xG.1'
]
displayxg_df = updated_data[columns_to_display]


In [100]:
displayxg_df.tail(10)

,Match ID,Team 1,Team 2,team_1_xg,team_2_xg,Avg xG Team 1,Avg xG Team 2,Weighted Avg xG Team 1,Weighted Avg xG Team 2,xG,xG.1,Avg xG,Avg xG.1,Weighted Avg xG,Weighted Avg xG.1
0,497410,Manchester Utd,Fulham,2.042680,0.418711,0.000000,0.000000,0.000000,0.000000,2.4,0.4,0.00,0.000000,0.00,0.000000
1,497412,Arsenal,Wolves,1.628300,0.575835,0.000000,0.000000,0.000000,0.000000,1.2,0.5,0.00,0.000000,0.00,0.000000
2,497436,Manchester Utd,Liverpool,1.500870,2.103970,2.042680,0.000000,7.149380,0.000000,1.4,1.8,2.40,0.000000,8.40,0.000000
3,497447,Southampton,Manchester Utd,1.355930,3.026930,0.000000,1.771775,0.000000,5.315325,1.1,2.6,0.00,1.900000,0.00,5.700000
4,497469,Wolves,Liverpool,0.748925,2.117240,0.575835,2.103970,2.879175,5.259925,0.6,2.5,0.50,1.800000,2.50,4.500000
5,497519,Wolves,Southampton,0.912781,0.792655,0.662380,1.355930,2.649520,3.389825,1.3,0.6,0.55,1.100000,2.20,2.750000
6,497541,Arsenal,Manchester Utd,2.966840,0.256083,1.628300,2.190160,4.070750,7.665560,2.1,0.2,1.20,2.133333,3.00,7.466667
7,497577,Manchester Utd,Bournemouth,NaN,NaN,1.706641,0.000000,5.461250,0.000000,NaN,NaN,1.65,0.000000,5.28,0.000000


In [101]:
def calculate_weighted_stats_xp(df):
    stats = ['np_xg', 'expected_points']  # Unified stat names
    cumulative = {stat: {} for stat in stats}  # Unified cumulative stats
    games_played = {}

    # Initialize result columns
    for stat in stats:
        df[f'Cum {stat}'] = 0  # Unified cumulative for each team
        df[f'Avg team1_{stat}'] = 0.0
        df[f'Avg team2_{stat}'] = 0.0
        df[f'Weighted Avg team1_{stat}'] = 0.0
        df[f'Weighted Avg team2_{stat}'] = 0.0

    for index, row in df.iterrows():
        team1 = row["Team 1"]
        team2 = row["Team 2"]

        # Initialize teams in cumulative and games_played if not already present
        for team in [team1, team2]:
            if team not in games_played:
                games_played[team] = 0
                for stat in stats:
                    cumulative[stat][team] = 0.0

        for stat in stats:
            # Process for team1
            if games_played[team1] > 0:  # Exclude current match stats
                avg_team1 = cumulative[stat][team1] / games_played[team1]
                fdr_team1 = row['Avg Team 1 FDR']
                weighted_avg_team1 = avg_team1 * fdr_team1

                df.at[index, f'Avg team1_{stat}'] = avg_team1
                df.at[index, f'Weighted Avg team1_{stat}'] = weighted_avg_team1
            else:
                df.at[index, f'Avg team1_{stat}'] = 0
                df.at[index, f'Weighted Avg team1_{stat}'] = 0

            # Process for team2
            if games_played[team2] > 0:  # Exclude current match stats
                avg_team2 = cumulative[stat][team2] / games_played[team2]
                fdr_team2 = row['Avg Team 2 FDR']
                weighted_avg_team2 = avg_team2 * fdr_team2

                df.at[index, f'Avg team2_{stat}'] = avg_team2
                df.at[index, f'Weighted Avg team2_{stat}'] = weighted_avg_team2
            else:
                df.at[index, f'Avg team2_{stat}'] = 0
                df.at[index, f'Weighted Avg team2_{stat}'] = 0

            # Update cumulative stats AFTER calculating averages
            team1_value = row[f'team1_{stat}'] if pd.notna(row[f'team1_{stat}']) else 0
            team2_value = row[f'team2_{stat}'] if pd.notna(row[f'team2_{stat}']) else 0

            cumulative[stat][team1] += team1_value
            cumulative[stat][team2] += team2_value

            # Unified cumulative (including current stats)
            df.at[index, f'Cum {stat}'] = cumulative[stat][team1] + cumulative[stat][team2]

        # Increment games played for both teams AFTER calculation
        games_played[team1] += 1
        games_played[team2] += 1

    return df


In [102]:
# Call the function to calculate stats
processed_data = calculate_weighted_stats_xp(data_df)

# Define the columns to display for checking
columns_to_display = [
    'Match ID', 'Team 1', 'Team 2',
    'team1_np_xg', 'team2_np_xg', 'Avg team1_np_xg', 'Avg team2_np_xg',
    'team1_expected_points', 'team2_expected_points',  'Avg team1_expected_points','Avg team2_expected_points', 'team_1_xg', 'team_2_xg',
    'Avg xG Team 1', 'Avg xG Team 2',
    'Weighted Avg xG Team 1', 'Weighted Avg xG Team 2',  'xG', 'xG.1','Avg xG','Avg xG.1', 'Weighted Avg xG', 'Weighted Avg xG.1'
]

# Display the filtered DataFrame
processed_data[columns_to_display].tail(10) # Adjust the number of rows to inspect more games


/var/folders/dy/ysjjzh0121qb38fzyck4zbdh0000gn/T/ipykernel_35742/1574674132.py:58: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2.461391' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, f'Cum {stat}'] = cumulative[stat][team1] + cumulative[stat][team2]
/var/folders/dy/ysjjzh0121qb38fzyck4zbdh0000gn/T/ipykernel_35742/1574674132.py:58: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2.8510999999999997' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, f'Cum {stat}'] = cumulative[stat][team1] + cumulative[stat][team2]


,Match ID,Team 1,Team 2,team1_np_xg,team2_np_xg,Avg team1_np_xg,Avg team2_np_xg,team1_expected_points,team2_expected_points,Avg team1_expected_points,Avg team2_expected_points,team_1_xg,team_2_xg,Avg xG Team 1,Avg xG Team 2,Weighted Avg xG Team 1,Weighted Avg xG Team 2,xG,xG.1,Avg xG,Avg xG.1,Weighted Avg xG,Weighted Avg xG.1
0,497410,Manchester Utd,Fulham,2.042680,0.418711,0.000000,0.000000,2.5696,0.2815,0.00000,0.00000,2.042680,0.418711,0.000000,0.000000,0.000000,0.000000,2.4,0.4,0.00,0.000000,0.00,0.000000
1,497412,Arsenal,Wolves,1.628300,0.575835,0.000000,0.000000,2.1894,0.5796,0.00000,0.00000,1.628300,0.575835,0.000000,0.000000,0.000000,0.000000,1.2,0.5,0.00,0.000000,0.00,0.000000
2,497436,Manchester Utd,Liverpool,1.500870,2.103970,2.042680,0.000000,0.9689,1.7849,2.56960,0.00000,1.500870,2.103970,2.042680,0.000000,7.149380,0.000000,1.4,1.8,2.40,0.000000,8.40,0.000000
3,497447,Southampton,Manchester Utd,0.545317,3.026930,0.000000,1.771775,0.3435,2.5032,0.00000,1.76925,1.355930,3.026930,0.000000,1.771775,0.000000,5.315325,1.1,2.6,0.00,1.900000,0.00,5.700000
4,497469,Wolves,Liverpool,0.748925,1.356070,0.575835,2.103970,0.3471,2.4807,0.57960,1.78490,0.748925,2.117240,0.575835,2.103970,2.879175,5.259925,0.6,2.5,0.50,1.800000,2.50,4.500000
5,497519,Wolves,Southampton,0.912781,0.792655,0.662380,0.545317,1.4630,1.1777,0.46335,0.34350,0.912781,0.792655,0.662380,1.355930,2.649520,3.389825,1.3,0.6,0.55,1.100000,2.20,2.750000
6,497541,Arsenal,Manchester Utd,2.966840,0.256083,1.628300,2.190160,2.9395,0.0343,2.18940,2.01390,2.966840,0.256083,1.628300,2.190160,4.070750,7.665560,2.1,0.2,1.20,2.133333,3.00,7.466667
7,497577,Manchester Utd,Bournemouth,NaN,NaN,1.706641,0.000000,NaN,NaN,1.51900,0.00000,NaN,NaN,1.706641,0.000000,5.461250,0.000000,NaN,NaN,1.65,0.000000,5.28,0.000000


### Save the data

In [104]:
processed_data.to_csv("updated_data.csv")

### FE Average win/draw/points/goalscored ... stats

In [106]:
import pandas as pd

def calculate_cumulative_stats(dataframe):
    # List of stats to track
    stats_columns = [
        'Won', 'Drawn', 'Lost', 'Goals Scored', 'Goals Conceded', 
        'Goal Difference', 'Points'
    ]

    # Ensure the necessary columns exist
    required_columns = [f'Prev Team 1 {stat}' for stat in stats_columns] + [f'Prev Team 2 {stat}' for stat in stats_columns]
    missing_columns = [col for col in required_columns if col not in dataframe.columns]
    if missing_columns:
        raise ValueError(f"Missing columns in DataFrame: {missing_columns}")

    # Initialize dictionaries to track cumulative stats and games played
    cumulative_stats = {}
    games_played = {}

    # Initialize columns in the DataFrame to store cumulative and average stats for each team
    for stat in stats_columns:
        dataframe[f'Cum Team 1 {stat}'] = 0
        dataframe[f'Cum Team 2 {stat}'] = 0
        dataframe[f'Avg Team 1 {stat}'] = 0.0
        dataframe[f'Avg Team 2 {stat}'] = 0.0

    # Lists to store the values temporarily
    temp_cum_team1 = {stat: [] for stat in stats_columns}
    temp_cum_team2 = {stat: [] for stat in stats_columns}
    temp_avg_team1 = {stat: [] for stat in stats_columns}
    temp_avg_team2 = {stat: [] for stat in stats_columns}

    # Iterate over rows
    for index, row in dataframe.iterrows():
        team1 = row['Team 1']
        team2 = row['Team 2']

        if team1 not in cumulative_stats:
            cumulative_stats[team1] = {stat: 0 for stat in stats_columns}
            games_played[team1] = 0
        if team2 not in cumulative_stats:
            cumulative_stats[team2] = {stat: 0 for stat in stats_columns}
            games_played[team2] = 0

        for stat in stats_columns:
            # Update cumulative stats
            cumulative_stats[team1][stat] += row[f'Prev Team 1 {stat}']
            cumulative_stats[team2][stat] += row[f'Prev Team 2 {stat}']

        # Calculate averages using the count of games played so far
        for stat in stats_columns:
            if games_played[team1] > 0:
                avg_team_1_stat = cumulative_stats[team1][stat] / games_played[team1]
                temp_avg_team1[stat].append(avg_team_1_stat)
                temp_cum_team1[stat].append(cumulative_stats[team1][stat])
            else:
                temp_avg_team1[stat].append(0)
                temp_cum_team1[stat].append(0)
            
            if games_played[team2] > 0:
                avg_team_2_stat = cumulative_stats[team2][stat] / games_played[team2]
                temp_avg_team2[stat].append(avg_team_2_stat)
                temp_cum_team2[stat].append(cumulative_stats[team2][stat])
            else:
                temp_avg_team2[stat].append(0)
                temp_cum_team2[stat].append(0)

        # Increment games played after calculation
        games_played[team1] += 1
        games_played[team2] += 1

    # Assign lists to DataFrame columns
    for stat in stats_columns:
        dataframe[f'Cum Team 1 {stat}'] = temp_cum_team1[stat]
        dataframe[f'Cum Team 2 {stat}'] = temp_cum_team2[stat]
        dataframe[f'Avg Team 1 {stat}'] = temp_avg_team1[stat]
        dataframe[f'Avg Team 2 {stat}'] = temp_avg_team2[stat]

    return dataframe

# Example usage assuming 'data' is your DataFrame loaded from 'your_data.csv'
try:
    updated_data = calculate_cumulative_stats(processed_data)
except ValueError as e:
    print(e)


/var/folders/dy/ysjjzh0121qb38fzyck4zbdh0000gn/T/ipykernel_35742/2096320922.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dataframe[f'Avg Team 2 {stat}'] = 0.0


In [107]:
updated_data[['Match ID', 'Team 1', 'Team 2'] + [col for col in updated_data.columns if 'Avg' in col or 'Cum' in col]].head()

,Match ID,Team 1,Team 2,Cum Team 1 FDR,Cum Team 2 FDR,Avg Team 1 FDR,Avg Team 2 FDR,Cum PPDA Team 1,Cum PPDA Team 2,Avg PPDA Team 1,Avg PPDA Team 2,Cum Deep Completions Team 1,Cum Deep Completions Team 2,Avg Deep Completions Team 1,Avg Deep Completions Team 2,Weighted Avg PPDA Team 1,Weighted Avg PPDA Team 2,Weighted Avg Deep Completions Team 1,Weighted Avg Deep Completions Team 2,Cum xG,Avg xG,Weighted Avg xG,Cum xG.1,Avg xG.1,Weighted Avg xG.1,Cum xG Team 1,Cum xG Team 2,Avg xG Team 1,Avg xG Team 2,Weighted Avg xG Team 1,Weighted Avg xG Team 2,Cum np_xg,Avg team1_np_xg,Avg team2_np_xg,Weighted Avg team1_np_xg,Weighted Avg team2_np_xg,Cum expected_points,Avg team1_expected_points,Avg team2_expected_points,Weighted Avg team1_expected_points,Weighted Avg team2_expected_points,Cum Team 1 Won,Cum Team 2 Won,Avg Team 1 Won,Avg Team 2 Won,Cum Team 1 Drawn,Cum Team 2 Drawn,Avg Team 1 Drawn,Avg Team 2 Drawn,Cum Team 1 Lost,Cum Team 2 Lost,Avg Team 1 Lost,Avg Team 2 Lost,Cum Team 1 Goals Scored,Cum Team 2 Goals Scored,Avg Team 1 Goals Scored,Avg Team 2 Goals Scored,Cum Team 1 Goals Conceded,Cum Team 2 Goals Conceded,Avg Team 1 Goals Conceded,Avg Team 2 Goals Conceded,Cum Team 1 Goal Difference,Cum Team 2 Goal Difference,Avg Team 1 Goal Difference,Avg Team 2 Goal Difference,Cum Team 1 Points,Cum Team 2 Points,Avg Team 1 Points,Avg Team 2 Points
0,497410,Manchester Utd,Fulham,2,3,2.0,3.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0,0.0,0.0,0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,2.461391,0.000000,0.000000,0.000000,0.000000,2.8511,0.0000,0.00000,0.0000,0.00000,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0
1,497412,Arsenal,Wolves,2,5,2.0,5.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0,0.0,0.0,0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,2.204135,0.000000,0.000000,0.000000,0.000000,2.7690,0.0000,0.00000,0.0000,0.00000,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0
2,497436,Manchester Utd,Liverpool,7,3,3.5,3.0,7.379310,0.000000,7.379310,0.000000,7.0,0.0,7.0,0.0,14.758621,0.000000,14.0,0.00,0,2.4,8.4,0,0.0,0.0,2.042680,0.00000,2.042680,0.000000,7.149380,0.000000,5.647520,2.042680,0.000000,7.149380,0.000000,5.3234,2.5696,0.00000,8.9936,0.00000,1,0,1.0,0.0,0,0,0.0,0.0,1,0,1.0,0.0,2,0,2.0,0.0,2,0,2.0,0.0,0,0,0.0,0.0,3,0,3.0,0.0
3,497447,Southampton,Manchester Utd,3,9,3.0,3.0,0.000000,19.879310,0.000000,9.939655,0.0,13.0,0.0,6.5,0.000000,34.788793,0.0,22.75,0,0.0,0.0,0,1.9,5.7,0.000000,3.54355,0.000000,1.771775,0.000000,5.315325,7.115797,0.000000,1.771775,0.000000,5.315325,6.3852,0.0000,1.76925,0.0000,5.30775,0,2,0.0,1.0,0,0,0.0,0.0,0,3,0.0,1.5,0,4,0.0,2.0,0,7,0.0,3.5,0,-3,0.0,-1.5,0,6,0.0,3.0
4,497469,Wolves,Liverpool,10,5,5.0,2.5,10.818182,11.692308,10.818182,11.692308,2.0,9.0,2.0,9.0,54.090909,35.076923,10.0,27.00,0,0.5,2.5,0,1.8,4.5,0.575835,2.10397,0.575835,2.103970,2.879175,5.259925,4.784800,0.575835,2.103970,2.879175,5.259925,5.1923,0.5796,1.78490,2.8980,4.46225,0,6,0.0,6.0,1,0,1.0,0.0,4,1,4.0,1.0,5,14,5.0,14.0,14,1,14.0,1.0,-9,13,-9.0,13.0,1,18,1.0,18.0


In [108]:
updated_data.head()

,Match ID,Team 1,Team 2,Team 1 Score,Team 2 Score,Winner,Prev Team 1 Played,Prev Team 1 Won,Prev Team 1 Drawn,Prev Team 1 Lost,Prev Team 1 Goals Scored,Prev Team 1 Goals Conceded,Prev Team 1 Goal Difference,Prev Team 1 Points,Prev Team 1 Form,Prev Team 2 Played,Prev Team 2 Won,Prev Team 2 Drawn,Prev Team 2 Lost,Prev Team 2 Goals Scored,Prev Team 2 Goals Conceded,Prev Team 2 Goal Difference,Prev Team 2 Points,Prev Team 2 Form,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Home_Total_Wins_Last_4,H2H_Away_Total_Wins_Last_4,H2H_Draws_Last_4,Composite Key,team_2_code,team_1_code,team_1_xg,team_2_xg,is_result,has_data,url,team2_points,team2_expected_points,team2_np_xg,team2_np_xg_difference,team2_ppda,team2_deep_completions,team1_points,team1_expected_points,team1_np_xg,team1_np_xg_difference,team1_ppda,team1_deep_completions,Unnamed: 0_x,gameweek,team_1_fdr,team_2_fdr,composite_key,B365H,B365D,B365A,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,team1_player_average,team2_player_average,Unnamed: 0,Score,Away Win Odds,Home Win Odds,Draw Odds,Precipitation,Cum Team 1 FDR,Cum Team 2 FDR,Avg Team 1 FDR,Avg Team 2 FDR,Cum PPDA Team 1,Cum PPDA Team 2,Avg PPDA Team 1,Avg PPDA Team 2,Cum Deep Completions Team 1,Cum Deep Completions Team 2,Avg Deep Completions Team 1,Avg Deep Completions Team 2,Weighted Avg PPDA Team 1,Weighted Avg PPDA Team 2,Weighted Avg Deep Completions Team 1,Weighted Avg Deep Completions Team 2,Cum xG,Avg xG,Weighted Avg xG,Cum xG.1,Avg xG.1,Weighted Avg xG.1,Cum xG Team 1,Cum xG Team 2,Avg xG Team 1,Avg xG Team 2,Weighted Avg xG Team 1,Weighted Avg xG Team 2,Cum np_xg,Avg team1_np_xg,Avg team2_np_xg,Weighted Avg team1_np_xg,Weighted Avg team2_np_xg,Cum expected_points,Avg team1_expected_points,Avg team2_expected_points,Weighted Avg team1_expected_points,Weighted Avg team2_expected_points,Cum Team 1 Won,Cum Team 2 Won,Avg Team 1 Won,Avg Team 2 Won,Cum Team 1 Drawn,Cum Team 2 Drawn,Avg Team 1 Drawn,Avg Team 2 Drawn,Cum Team 1 Lost,Cum Team 2 Lost,Avg Team 1 Lost,Avg Team 2 Lost,Cum Team 1 Goals Scored,Cum Team 2 Goals Scored,Avg Team 1 Goals Scored,Avg Team 2 Goals Scored,Cum Team 1 Goals Conceded,Cum Team 2 Goals Conceded,Avg Team 1 Goals Conceded,Avg Team 2 Goals Conceded,Cum Team 1 Goal Difference,Cum Team 2 Goal Difference,Avg Team 1 Goal Difference,Avg Team 2 Goal Difference,Cum Team 1 Points,Cum Team 2 Points,Avg Team 1 Points,Avg Team 2 Points
0,497410,Manchester Utd,Fulham,1.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-16,19:00,1,Fri,2.4,0.4,73297.0,Old Trafford,Robert Jones,1.0,53.4631,-2.2914,2024-08-16T19:00:00,14.313000,1,0,0,2024-08-16_Manchester Utd_Fulham,FLH,MUN,2.042680,0.418711,True,True,https://understat.com/match/26602,0.0,0.2815,0.418711,-1.623969,10.833333,3.0,3.0,2.5696,2.042680,1.623969,7.379310,7.0,1,1.0,2,3,2024-08-16_Manchester Utd_Fulham,1.60,4.20,5.25,5.0,2.0,12.0,10.0,7.0,8.0,2.0,3.0,0.0,0.0,9.143500,8.974790,NaN,NaN,NaN,NaN,NaN,NaN,2,3,2.0,3.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0,0.0,0.0,0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,2.461391,0.000000,0.000000,0.000000,0.000000,2.8511,0.0000,0.00000,0.0000,0.00000,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0
1,497412,Arsenal,Wolves,2.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00,1,Sat,1.2,0.5,60261.0,Emirates Stadium,Jarred Gillett,3.0,51.5550,-0.1086,2024-08-17T14:00:00,21.734999,3,0,0,2024-08-17_Arsenal_Wolves,WOL,ARS,1.628300,0.575835,True,True,https://understat.com/match/26604,0.0,0.5796,0.575835,-1.052465,10.818182,2.0,3.0,2.1894,1.628300,1.052465,7.769231,14.0,3,1.0,2,5,2024-08-17_Arsenal_Wolves,1.18,7.50,13.00,6.0,3.0,17.0,14.0,8.0,2.0,2.0,2.0,0.0,0.0,10.830699,6.027272,NaN,NaN,NaN,NaN,NaN,NaN,2,5,2.0,5.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0,0.0,0.0,0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.0

#### Categorize date/time

In [110]:
data = updated_data

In [111]:
data.head()

,Match ID,Team 1,Team 2,Team 1 Score,Team 2 Score,Winner,Prev Team 1 Played,Prev Team 1 Won,Prev Team 1 Drawn,Prev Team 1 Lost,Prev Team 1 Goals Scored,Prev Team 1 Goals Conceded,Prev Team 1 Goal Difference,Prev Team 1 Points,Prev Team 1 Form,Prev Team 2 Played,Prev Team 2 Won,Prev Team 2 Drawn,Prev Team 2 Lost,Prev Team 2 Goals Scored,Prev Team 2 Goals Conceded,Prev Team 2 Goal Difference,Prev Team 2 Points,Prev Team 2 Form,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Home_Total_Wins_Last_4,H2H_Away_Total_Wins_Last_4,H2H_Draws_Last_4,Composite Key,team_2_code,team_1_code,team_1_xg,team_2_xg,is_result,has_data,url,team2_points,team2_expected_points,team2_np_xg,team2_np_xg_difference,team2_ppda,team2_deep_completions,team1_points,team1_expected_points,team1_np_xg,team1_np_xg_difference,team1_ppda,team1_deep_completions,Unnamed: 0_x,gameweek,team_1_fdr,team_2_fdr,composite_key,B365H,B365D,B365A,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,team1_player_average,team2_player_average,Unnamed: 0,Score,Away Win Odds,Home Win Odds,Draw Odds,Precipitation,Cum Team 1 FDR,Cum Team 2 FDR,Avg Team 1 FDR,Avg Team 2 FDR,Cum PPDA Team 1,Cum PPDA Team 2,Avg PPDA Team 1,Avg PPDA Team 2,Cum Deep Completions Team 1,Cum Deep Completions Team 2,Avg Deep Completions Team 1,Avg Deep Completions Team 2,Weighted Avg PPDA Team 1,Weighted Avg PPDA Team 2,Weighted Avg Deep Completions Team 1,Weighted Avg Deep Completions Team 2,Cum xG,Avg xG,Weighted Avg xG,Cum xG.1,Avg xG.1,Weighted Avg xG.1,Cum xG Team 1,Cum xG Team 2,Avg xG Team 1,Avg xG Team 2,Weighted Avg xG Team 1,Weighted Avg xG Team 2,Cum np_xg,Avg team1_np_xg,Avg team2_np_xg,Weighted Avg team1_np_xg,Weighted Avg team2_np_xg,Cum expected_points,Avg team1_expected_points,Avg team2_expected_points,Weighted Avg team1_expected_points,Weighted Avg team2_expected_points,Cum Team 1 Won,Cum Team 2 Won,Avg Team 1 Won,Avg Team 2 Won,Cum Team 1 Drawn,Cum Team 2 Drawn,Avg Team 1 Drawn,Avg Team 2 Drawn,Cum Team 1 Lost,Cum Team 2 Lost,Avg Team 1 Lost,Avg Team 2 Lost,Cum Team 1 Goals Scored,Cum Team 2 Goals Scored,Avg Team 1 Goals Scored,Avg Team 2 Goals Scored,Cum Team 1 Goals Conceded,Cum Team 2 Goals Conceded,Avg Team 1 Goals Conceded,Avg Team 2 Goals Conceded,Cum Team 1 Goal Difference,Cum Team 2 Goal Difference,Avg Team 1 Goal Difference,Avg Team 2 Goal Difference,Cum Team 1 Points,Cum Team 2 Points,Avg Team 1 Points,Avg Team 2 Points
0,497410,Manchester Utd,Fulham,1.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-16,19:00,1,Fri,2.4,0.4,73297.0,Old Trafford,Robert Jones,1.0,53.4631,-2.2914,2024-08-16T19:00:00,14.313000,1,0,0,2024-08-16_Manchester Utd_Fulham,FLH,MUN,2.042680,0.418711,True,True,https://understat.com/match/26602,0.0,0.2815,0.418711,-1.623969,10.833333,3.0,3.0,2.5696,2.042680,1.623969,7.379310,7.0,1,1.0,2,3,2024-08-16_Manchester Utd_Fulham,1.60,4.20,5.25,5.0,2.0,12.0,10.0,7.0,8.0,2.0,3.0,0.0,0.0,9.143500,8.974790,NaN,NaN,NaN,NaN,NaN,NaN,2,3,2.0,3.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0,0.0,0.0,0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,2.461391,0.000000,0.000000,0.000000,0.000000,2.8511,0.0000,0.00000,0.0000,0.00000,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0
1,497412,Arsenal,Wolves,2.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00,1,Sat,1.2,0.5,60261.0,Emirates Stadium,Jarred Gillett,3.0,51.5550,-0.1086,2024-08-17T14:00:00,21.734999,3,0,0,2024-08-17_Arsenal_Wolves,WOL,ARS,1.628300,0.575835,True,True,https://understat.com/match/26604,0.0,0.5796,0.575835,-1.052465,10.818182,2.0,3.0,2.1894,1.628300,1.052465,7.769231,14.0,3,1.0,2,5,2024-08-17_Arsenal_Wolves,1.18,7.50,13.00,6.0,3.0,17.0,14.0,8.0,2.0,2.0,2.0,0.0,0.0,10.830699,6.027272,NaN,NaN,NaN,NaN,NaN,NaN,2,5,2.0,5.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0,0.0,0.0,0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.0

In [112]:
import pandas as pd
from datetime import datetime

# Parse date and time columns
data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d')  # Corrected format
data['Time'] = pd.to_datetime(data['Time'], format='%H:%M').dt.time

# Function to determine sunset time based on season
def get_sunset_time(date):
    month = date.month
    if month in [12, 1, 2]:  # Winter
        return datetime.strptime("16:00", "%H:%M").time()
    elif month in [3, 4, 5]:  # Spring
        return datetime.strptime("20:30", "%H:%M").time() if month == 5 else datetime.strptime("19:30", "%H:%M").time() if month == 4 else datetime.strptime("18:30", "%H:%M").time()
    elif month in [6, 7, 8]:  # Summer
        return datetime.strptime("21:30", "%H:%M").time()
    elif month in [9, 10, 11]:  # Autumn
        return datetime.strptime("18:30", "%H:%M").time() if month == 9 else datetime.strptime("17:00", "%H:%M").time() if month == 10 else datetime.strptime("16:30", "%H:%M").time()

# Calculate sunset times
sunset_times = data['Date'].apply(get_sunset_time)

# Categorize match as daytime or nighttime
def categorize_match(match_time, sunset_time):
    return 'Nighttime' if match_time > sunset_time else 'Daytime'

match_categories = data.apply(lambda row: categorize_match(row['Time'], sunset_times.loc[row.name]), axis=1)

# Assign computed series to DataFrame all at once
data = data.assign(Sunset_Time=sunset_times, Match_Time_Category=match_categories)

# Display the resulting DataFrame
print(data[['Date', 'Time', 'Sunset_Time', 'Match_Time_Category']])


        Date      Time Sunset_Time Match_Time_Category
0 2024-08-16  19:00:00    21:30:00             Daytime
1 2024-08-17  14:00:00    21:30:00             Daytime
2 2024-09-01  15:00:00    18:30:00             Daytime
3 2024-09-14  11:30:00    18:30:00             Daytime
4 2024-09-28  16:30:00    18:30:00             Daytime
5 2024-11-09  15:00:00    16:30:00             Daytime
6 2024-12-04  20:15:00    16:00:00           Nighttime
7 2024-12-22  14:00:00    16:00:00             Daytime


In [113]:
data.head(30)

,Match ID,Team 1,Team 2,Team 1 Score,Team 2 Score,Winner,Prev Team 1 Played,Prev Team 1 Won,Prev Team 1 Drawn,Prev Team 1 Lost,Prev Team 1 Goals Scored,Prev Team 1 Goals Conceded,Prev Team 1 Goal Difference,Prev Team 1 Points,Prev Team 1 Form,Prev Team 2 Played,Prev Team 2 Won,Prev Team 2 Drawn,Prev Team 2 Lost,Prev Team 2 Goals Scored,Prev Team 2 Goals Conceded,Prev Team 2 Goal Difference,Prev Team 2 Points,Prev Team 2 Form,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Home_Total_Wins_Last_4,H2H_Away_Total_Wins_Last_4,H2H_Draws_Last_4,Composite Key,team_2_code,team_1_code,team_1_xg,team_2_xg,is_result,has_data,url,team2_points,team2_expected_points,team2_np_xg,team2_np_xg_difference,team2_ppda,team2_deep_completions,team1_points,team1_expected_points,team1_np_xg,team1_np_xg_difference,team1_ppda,team1_deep_completions,Unnamed: 0_x,gameweek,team_1_fdr,team_2_fdr,composite_key,B365H,B365D,B365A,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,team1_player_average,team2_player_average,Unnamed: 0,Score,Away Win Odds,Home Win Odds,Draw Odds,Precipitation,Cum Team 1 FDR,Cum Team 2 FDR,Avg Team 1 FDR,Avg Team 2 FDR,Cum PPDA Team 1,Cum PPDA Team 2,Avg PPDA Team 1,Avg PPDA Team 2,Cum Deep Completions Team 1,Cum Deep Completions Team 2,Avg Deep Completions Team 1,Avg Deep Completions Team 2,Weighted Avg PPDA Team 1,Weighted Avg PPDA Team 2,Weighted Avg Deep Completions Team 1,Weighted Avg Deep Completions Team 2,Cum xG,Avg xG,Weighted Avg xG,Cum xG.1,Avg xG.1,Weighted Avg xG.1,Cum xG Team 1,Cum xG Team 2,Avg xG Team 1,Avg xG Team 2,Weighted Avg xG Team 1,Weighted Avg xG Team 2,Cum np_xg,Avg team1_np_xg,Avg team2_np_xg,Weighted Avg team1_np_xg,Weighted Avg team2_np_xg,Cum expected_points,Avg team1_expected_points,Avg team2_expected_points,Weighted Avg team1_expected_points,Weighted Avg team2_expected_points,Cum Team 1 Won,Cum Team 2 Won,Avg Team 1 Won,Avg Team 2 Won,Cum Team 1 Drawn,Cum Team 2 Drawn,Avg Team 1 Drawn,Avg Team 2 Drawn,Cum Team 1 Lost,Cum Team 2 Lost,Avg Team 1 Lost,Avg Team 2 Lost,Cum Team 1 Goals Scored,Cum Team 2 Goals Scored,Avg Team 1 Goals Scored,Avg Team 2 Goals Scored,Cum Team 1 Goals Conceded,Cum Team 2 Goals Conceded,Avg Team 1 Goals Conceded,Avg Team 2 Goals Conceded,Cum Team 1 Goal Difference,Cum Team 2 Goal Difference,Avg Team 1 Goal Difference,Avg Team 2 Goal Difference,Cum Team 1 Points,Cum Team 2 Points,Avg Team 1 Points,Avg Team 2 Points,Sunset_Time,Match_Time_Category
0,497410,Manchester Utd,Fulham,1.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-16,19:00:00,1,Fri,2.4,0.4,73297.0,Old Trafford,Robert Jones,1.0,53.4631,-2.2914,2024-08-16T19:00:00,14.313000,1,0,0,2024-08-16_Manchester Utd_Fulham,FLH,MUN,2.042680,0.418711,True,True,https://understat.com/match/26602,0.0,0.2815,0.418711,-1.623969,10.833333,3.0,3.0,2.5696,2.042680,1.623969,7.379310,7.0,1,1.0,2,3,2024-08-16_Manchester Utd_Fulham,1.60,4.20,5.25,5.0,2.0,12.0,10.0,7.0,8.0,2.0,3.0,0.0,0.0,9.143500,8.974790,NaN,NaN,NaN,NaN,NaN,NaN,2,3,2.0,3.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.0,0.000000,0.000000,0.000,0.00,0,0.00,0.00,0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,2.461391,0.000000,0.000000,0.000000,0.000000,2.8511,0.00000,0.00000,0.0000,0.00000,0,0,0.0,0.000000,0,0,0.0,0.000000,0,0,0.00,0.000000,0,0,0.0,0.0,0,0,0.0,0.000000,0,0,0.0,0.000000,0,0,0.0,0.000000,21:30:00,Daytime
1,497412,Arsenal,Wolves,2.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00:00,1,Sat,1.2,0.5,60261.0,Emirates Stadium,Jarred Gillett,3.0,51.5550,-0.1086,2024-08-17T14:00:00,21.734999,3,0,0,2024-08-17_Arsenal_Wolves,WOL,ARS,1.628300,0.575835,True,True,https://understat.com/match/26604,0.0,0.5796,0.575835,-1.052465,10.818182,2.0,3.0,2.1894,1.628300,1.052465,7.769231,14.0,3,1.0,2,5,2024-08-17_Arsenal_Wolves,1.18,7.50,13.00,6.0,3.0,17.0,14.0,8.0,2.0,2.0,2.0,0.0,0.0,10.830699,6.027272,NaN,NaN,NaN,NaN,NaN,NaN,2,5,2.0,5.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,

#### PPG

In [115]:
import numpy as np

# Calculate points per game for Team 1
data['ppg_team1'] = np.where(
    data['Prev Team 1 Played'] > 0,
    data['Prev Team 1 Points'] / data['Prev Team 1 Played'],
    0
)

# Calculate points per game for Team 2
data['ppg_team2'] = np.where(
    data['Prev Team 2 Played'] > 0,
    data['Prev Team 2 Points'] / data['Prev Team 2 Played'],
    0
)


In [116]:
data.tail(30)

,Match ID,Team 1,Team 2,Team 1 Score,Team 2 Score,Winner,Prev Team 1 Played,Prev Team 1 Won,Prev Team 1 Drawn,Prev Team 1 Lost,Prev Team 1 Goals Scored,Prev Team 1 Goals Conceded,Prev Team 1 Goal Difference,Prev Team 1 Points,Prev Team 1 Form,Prev Team 2 Played,Prev Team 2 Won,Prev Team 2 Drawn,Prev Team 2 Lost,Prev Team 2 Goals Scored,Prev Team 2 Goals Conceded,Prev Team 2 Goal Difference,Prev Team 2 Points,Prev Team 2 Form,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Home_Total_Wins_Last_4,H2H_Away_Total_Wins_Last_4,H2H_Draws_Last_4,Composite Key,team_2_code,team_1_code,team_1_xg,team_2_xg,is_result,has_data,url,team2_points,team2_expected_points,team2_np_xg,team2_np_xg_difference,team2_ppda,team2_deep_completions,team1_points,team1_expected_points,team1_np_xg,team1_np_xg_difference,team1_ppda,team1_deep_completions,Unnamed: 0_x,gameweek,team_1_fdr,team_2_fdr,composite_key,B365H,B365D,B365A,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,team1_player_average,team2_player_average,Unnamed: 0,Score,Away Win Odds,Home Win Odds,Draw Odds,Precipitation,Cum Team 1 FDR,Cum Team 2 FDR,Avg Team 1 FDR,Avg Team 2 FDR,Cum PPDA Team 1,Cum PPDA Team 2,Avg PPDA Team 1,Avg PPDA Team 2,Cum Deep Completions Team 1,Cum Deep Completions Team 2,Avg Deep Completions Team 1,Avg Deep Completions Team 2,Weighted Avg PPDA Team 1,Weighted Avg PPDA Team 2,Weighted Avg Deep Completions Team 1,Weighted Avg Deep Completions Team 2,Cum xG,Avg xG,Weighted Avg xG,Cum xG.1,Avg xG.1,Weighted Avg xG.1,Cum xG Team 1,Cum xG Team 2,Avg xG Team 1,Avg xG Team 2,Weighted Avg xG Team 1,Weighted Avg xG Team 2,Cum np_xg,Avg team1_np_xg,Avg team2_np_xg,Weighted Avg team1_np_xg,Weighted Avg team2_np_xg,Cum expected_points,Avg team1_expected_points,Avg team2_expected_points,Weighted Avg team1_expected_points,Weighted Avg team2_expected_points,Cum Team 1 Won,Cum Team 2 Won,Avg Team 1 Won,Avg Team 2 Won,Cum Team 1 Drawn,Cum Team 2 Drawn,Avg Team 1 Drawn,Avg Team 2 Drawn,Cum Team 1 Lost,Cum Team 2 Lost,Avg Team 1 Lost,Avg Team 2 Lost,Cum Team 1 Goals Scored,Cum Team 2 Goals Scored,Avg Team 1 Goals Scored,Avg Team 2 Goals Scored,Cum Team 1 Goals Conceded,Cum Team 2 Goals Conceded,Avg Team 1 Goals Conceded,Avg Team 2 Goals Conceded,Cum Team 1 Goal Difference,Cum Team 2 Goal Difference,Avg Team 1 Goal Difference,Avg Team 2 Goal Difference,Cum Team 1 Points,Cum Team 2 Points,Avg Team 1 Points,Avg Team 2 Points,Sunset_Time,Match_Time_Category,ppg_team1,ppg_team2
0,497410,Manchester Utd,Fulham,1.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-16,19:00:00,1,Fri,2.4,0.4,73297.0,Old Trafford,Robert Jones,1.0,53.4631,-2.2914,2024-08-16T19:00:00,14.313000,1,0,0,2024-08-16_Manchester Utd_Fulham,FLH,MUN,2.042680,0.418711,True,True,https://understat.com/match/26602,0.0,0.2815,0.418711,-1.623969,10.833333,3.0,3.0,2.5696,2.042680,1.623969,7.379310,7.0,1,1.0,2,3,2024-08-16_Manchester Utd_Fulham,1.60,4.20,5.25,5.0,2.0,12.0,10.0,7.0,8.0,2.0,3.0,0.0,0.0,9.143500,8.974790,NaN,NaN,NaN,NaN,NaN,NaN,2,3,2.0,3.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.0,0.000000,0.000000,0.000,0.00,0,0.00,0.00,0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,2.461391,0.000000,0.000000,0.000000,0.000000,2.8511,0.00000,0.00000,0.0000,0.00000,0,0,0.0,0.000000,0,0,0.0,0.000000,0,0,0.00,0.000000,0,0,0.0,0.0,0,0,0.0,0.000000,0,0,0.0,0.000000,0,0,0.0,0.000000,21:30:00,Daytime,0.000000,0.000000
1,497412,Arsenal,Wolves,2.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00:00,1,Sat,1.2,0.5,60261.0,Emirates Stadium,Jarred Gillett,3.0,51.5550,-0.1086,2024-08-17T14:00:00,21.734999,3,0,0,2024-08-17_Arsenal_Wolves,WOL,ARS,1.628300,0.575835,True,True,https://understat.com/match/26604,0.0,0.5796,0.575835,-1.052465,10.818182,2.0,3.0,2.1894,1.628300,1.052465,7.769231,14.0,3,1.0,2,5,2024-08-17_Arsenal_Wolves,1.18,7.50,13.00,6.0,3.0,17.0,14.0,8.0,2.0,2.0,2.0,0.0,0.0,10.830699,6.027272,NaN,NaN,NaN,NaN,NaN,NaN,2,5,2.0,5.0,0.0000

#### CUm Stats

In [118]:
# Initialize new columns for cumulative averages
stats = ['HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR']
for stat in stats:
    data[f'Cumulative_Avg_{stat}'] = 0.0

# Initialize dictionaries to track total stats and number of games for each team
team_stats_totals = {stat: {} for stat in stats}
team_game_counts = {stat: {} for stat in stats}

# Iterate through the DataFrame row by row
for index, row in data.iterrows():
    home_team = row['Team 1']
    away_team = row['Team 2']
    
    # Calculate cumulative averages for each stat
    for stat in stats:
        home_stat = stat  # Home version of the stat
        away_stat = stat  # Away version of the stat

        # Initialize stats for home and away teams if not already in dictionaries
        if home_team not in team_stats_totals[home_stat]:
            team_stats_totals[home_stat][home_team] = 0
            team_game_counts[home_stat][home_team] = 0
        if away_team not in team_stats_totals[away_stat]:
            team_stats_totals[away_stat][away_team] = 0
            team_game_counts[away_stat][away_team] = 0

        # Assign cumulative average to the new columns
        if team_game_counts[home_stat][home_team] > 0:
            data.at[index, f'Cumulative_Avg_{home_stat}'] = (
                team_stats_totals[home_stat][home_team] / team_game_counts[home_stat][home_team]
            )
        if team_game_counts[away_stat][away_team] > 0:
            data.at[index, f'Cumulative_Avg_{away_stat}'] = (
                team_stats_totals[away_stat][away_team] / team_game_counts[away_stat][away_team]
            )

        # Update totals and game counts with the current game's stats
        team_stats_totals[home_stat][home_team] += row[home_stat]
        team_game_counts[home_stat][home_team] += 1
        team_stats_totals[away_stat][away_team] += row[away_stat]
        team_game_counts[away_stat][away_team] += 1

# Display the updated DataFrame with cumulative averages
data.tail(13)


,Match ID,Team 1,Team 2,Team 1 Score,Team 2 Score,Winner,Prev Team 1 Played,Prev Team 1 Won,Prev Team 1 Drawn,Prev Team 1 Lost,Prev Team 1 Goals Scored,Prev Team 1 Goals Conceded,Prev Team 1 Goal Difference,Prev Team 1 Points,Prev Team 1 Form,Prev Team 2 Played,Prev Team 2 Won,Prev Team 2 Drawn,Prev Team 2 Lost,Prev Team 2 Goals Scored,Prev Team 2 Goals Conceded,Prev Team 2 Goal Difference,Prev Team 2 Points,Prev Team 2 Form,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Home_Total_Wins_Last_4,H2H_Away_Total_Wins_Last_4,H2H_Draws_Last_4,Composite Key,team_2_code,team_1_code,team_1_xg,team_2_xg,is_result,has_data,url,team2_points,team2_expected_points,team2_np_xg,team2_np_xg_difference,team2_ppda,team2_deep_completions,team1_points,team1_expected_points,team1_np_xg,team1_np_xg_difference,team1_ppda,team1_deep_completions,Unnamed: 0_x,gameweek,team_1_fdr,team_2_fdr,composite_key,B365H,B365D,B365A,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,team1_player_average,team2_player_average,Unnamed: 0,Score,Away Win Odds,Home Win Odds,Draw Odds,Precipitation,Cum Team 1 FDR,Cum Team 2 FDR,Avg Team 1 FDR,Avg Team 2 FDR,Cum PPDA Team 1,Cum PPDA Team 2,Avg PPDA Team 1,Avg PPDA Team 2,Cum Deep Completions Team 1,Cum Deep Completions Team 2,Avg Deep Completions Team 1,Avg Deep Completions Team 2,Weighted Avg PPDA Team 1,Weighted Avg PPDA Team 2,Weighted Avg Deep Completions Team 1,Weighted Avg Deep Completions Team 2,Cum xG,Avg xG,Weighted Avg xG,Cum xG.1,Avg xG.1,Weighted Avg xG.1,Cum xG Team 1,Cum xG Team 2,Avg xG Team 1,Avg xG Team 2,Weighted Avg xG Team 1,Weighted Avg xG Team 2,Cum np_xg,Avg team1_np_xg,Avg team2_np_xg,Weighted Avg team1_np_xg,Weighted Avg team2_np_xg,Cum expected_points,Avg team1_expected_points,Avg team2_expected_points,Weighted Avg team1_expected_points,Weighted Avg team2_expected_points,Cum Team 1 Won,Cum Team 2 Won,Avg Team 1 Won,Avg Team 2 Won,Cum Team 1 Drawn,Cum Team 2 Drawn,Avg Team 1 Drawn,Avg Team 2 Drawn,Cum Team 1 Lost,Cum Team 2 Lost,Avg Team 1 Lost,Avg Team 2 Lost,Cum Team 1 Goals Scored,Cum Team 2 Goals Scored,Avg Team 1 Goals Scored,Avg Team 2 Goals Scored,Cum Team 1 Goals Conceded,Cum Team 2 Goals Conceded,Avg Team 1 Goals Conceded,Avg Team 2 Goals Conceded,Cum Team 1 Goal Difference,Cum Team 2 Goal Difference,Avg Team 1 Goal Difference,Avg Team 2 Goal Difference,Cum Team 1 Points,Cum Team 2 Points,Avg Team 1 Points,Avg Team 2 Points,Sunset_Time,Match_Time_Category,ppg_team1,ppg_team2,Cumulative_Avg_HST,Cumulative_Avg_AST,Cumulative_Avg_HF,Cumulative_Avg_AF,Cumulative_Avg_HC,Cumulative_Avg_AC,Cumulative_Avg_HY,Cumulative_Avg_AY,Cumulative_Avg_HR,Cumulative_Avg_AR
0,497410,Manchester Utd,Fulham,1.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-16,19:00:00,1,Fri,2.4,0.4,73297.0,Old Trafford,Robert Jones,1.0,53.4631,-2.2914,2024-08-16T19:00:00,14.313000,1,0,0,2024-08-16_Manchester Utd_Fulham,FLH,MUN,2.042680,0.418711,True,True,https://understat.com/match/26602,0.0,0.2815,0.418711,-1.623969,10.833333,3.0,3.0,2.5696,2.042680,1.623969,7.379310,7.0,1,1.0,2,3,2024-08-16_Manchester Utd_Fulham,1.60,4.20,5.25,5.0,2.0,12.0,10.0,7.0,8.0,2.0,3.0,0.0,0.0,9.143500,8.974790,NaN,NaN,NaN,NaN,NaN,NaN,2,3,2.0,3.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.0,0.000000,0.000000,0.000,0.00,0,0.00,0.00,0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,2.461391,0.000000,0.000000,0.000000,0.000000,2.8511,0.00000,0.00000,0.0000,0.00000,0,0,0.0,0.000000,0,0,0.0,0.000000,0,0,0.00,0.000000,0,0,0.0,0.0,0,0,0.0,0.000000,0,0,0.0,0.000000,0,0,0.0,0.000000,21:30:00,Daytime,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0
1,497412,Arsenal,Wolves,2.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00:00,1,Sat,1.2,0.5,60261.0,Emirates Stadium,Jarred Gillett,3.0,51.5550,-0.1086,2024-08-17T14:00:00,21.734999,3,0,0,2024-08-17_Arsenal_Wolves,WOL,ARS,1.628300,0.575835,True,True,https://understat.com/match

In [119]:
data.to_csv("fact_table.csv")